In [ ]:
#IMPORT PYTHON PACKAGES
import seaborn as sns
import pandas as pd
import scipy as sp
import scipy.integrate
from scipy.stats import sem
import numpy  as np
import copy
import matplotlib.pylab as plt
from matplotlib import rcParams
from matplotlib.ticker import FormatStrFormatter
import time
import datetime
import subprocess
import itertools
from itertools import groupby, repeat, islice
import math as math
import decimal as d
from decimal import *

#%matplotlib notebook
#inline
#Plot settings
sns.set_style("whitegrid", rc={"axes.edgecolor": "k", "axes.linewidth":2.})

sns.set_style("ticks", {"xtick.major.size":8,"ytick.major.size":8})


sns.set_context("notebook",rc={"grid.linewidth": 0, 
                            "font.family":"Helvetica", "axes.labelsize":24.,"xtick.labelsize":24., 
                            "ytick.labelsize":24., "legend.fontsize":18.})

color_blind_safe = sns.color_palette("colorblind", 10) 

#colors = sns.color_palette("tab10", 10) 


In [ ]:
#200
filename = ['./profile_siE200r50','profile_siE200r125', 'profile_siE200r250', 'profile_siE200r271']

MP_Profile = []; MP_Ave = []; MP_LastN = []; 

for simNo in range(0,len(filename)):
    subprocess.call(r"perl -p -e 's/^[\ \t]+|[\ \t]+$//g' %s > %s_new" % (filename[simNo], filename[simNo]), shell=True) # remove trailling white spaces
    subprocess.call("mv %s_new %s" % (filename[simNo], filename[simNo]), shell=True)
    subprocess.call("tr -s ' ' < %s > %s_tmp " % (filename[simNo], filename[simNo]), shell=True) #squeeze spaces
    subprocess.call("mv %s_tmp %s" % (filename[simNo], filename[simNo]), shell=True)

    profile = []; N = 800;
    with open(filename[simNo], 'r') as f:
        while True:
            lines = list(itertools.islice(f, 1, N+1))
            for line in lines:
                profile.append(line.split(' '))
            if not lines: 
                break
                
    #lastN = profile[-N:] 
    lastAve = profile[-3200:]

    MP_Profile.append(profile)
    #MP_LastN.append(lastN)
    MP_Ave.append(lastAve)
    
MP_Profile = np.array(MP_Profile, dtype=float) # for plotting 3D graphs
#MP_LastN = np.array(MP_LastN, dtype=float) # for calculating the thermal conductivity
MP_Ave = np.array(MP_Ave, dtype=float) # for averaging the data to calculate the thermal conductivity


Thermo_Data = np.array([np.loadtxt('./Thermo_siE200r50'), np.loadtxt('Thermo_siE200r125'), np.loadtxt('Thermo_siE200r250'), np.loadtxt('Thermo_siE200r271')], dtype=object) #, np.loadtxt('Thermo_siE200r125_Original_2')])
                                                                      

labels = ['SiGeSi200r50','SiGeSi200r125', 'SiGeSi200r250', 'SiGeSi200r271'] 


for simNo in range(0, len(MP_Profile)):
    Coordinates = MP_Profile[simNo,:,1]*(Thermo_Data[simNo,-1,9])
    Temperature = MP_Profile[simNo,:,3]
   

# Load the data that will be averaged to calculate the slopes 

Temperature = []; Coordinate = []; dX_Thermo = [];

for simNo in range(0, len(filename)):
    
    dX_ = Thermo_Data[simNo,-1,9]
    dX_Thermo.append(dX_)
    
    for i in range(0,800):
        Temperature_ = np.mean([MP_Ave[simNo,i,3],MP_Ave[simNo,i+800,3],MP_Ave[simNo,i+1600,3],MP_Ave[simNo,i+2400,3]])   
        Temperature.append(Temperature_)
        Coordinate_ = np.mean([MP_Ave[simNo,i,1],MP_Ave[simNo,i+800,1],MP_Ave[simNo,i+1600,1],MP_Ave[simNo,i+2400,1]])*dX_
        Coordinate.append(Coordinate_)
    
Temperature = np.array([Temperature[0:800],Temperature[800:1600], Temperature[1600:2400], Temperature[2400:3200], 
                        ], dtype=float)
Coordinate = np.array([Coordinate[0:800],Coordinate[800:1600], Coordinate[1600:2400], Coordinate[2400:3200], 
                      ], dtype=float)


excluded_points1 = 230 # excluded points from the beginning
excluded_points2 = 40 # excluded points from the end
excluded_points3 = 20
excluded_points4 = 230 # from the end of the mp.data file

slope2_mean = []; slope2 = []; slope1_mean = []; slope1 = [];
slope3_mean = []; slope3 = []; slope4_mean = []; slope4 = [];
temp_D_200Si_both = []; temp_D_200Si = [];mid_temp_200Si = []; average_temp_200Si = [];
boundary_low_200Si = []; boundary_high_200Si = [];

for simNo in range(0,len(filename)):
    
    dTemp = Temperature[simNo,:]
    X = dX_
    dX = Coordinate[simNo,:] #*X # normalized distance
    
    T_fit_left = np.polyfit(dX[excluded_points1:400-excluded_points2], dTemp[excluded_points1:400-excluded_points2], 1)
    T_fit_right = -np.polyfit(dX[400+excluded_points2:-excluded_points1], dTemp[400+excluded_points2:-excluded_points1], 1)
    
    slope2.append([T_fit_left[0], T_fit_right[0]])
    
    mean_slope = np.mean([abs(T_fit_left[0]), abs(T_fit_right[0])], axis = 0)
    slope2_mean.append(mean_slope)
    
    B_fit_left2 = np.polyfit(dX[excluded_points3:400-excluded_points4], dTemp[excluded_points3:400-excluded_points4], 1)
    B_fit_right2 = -np.polyfit(dX[400+excluded_points4:-excluded_points3], dTemp[400+excluded_points4:-excluded_points3], 1)
    
    slope1.append([B_fit_left2[0], B_fit_right2[0]])
    
    mean_slope = np.mean([abs(B_fit_left2[0]), abs(B_fit_right2[0])], axis = 0)
    slope1_mean.append(mean_slope)
    
    slope3.append([T_fit_left[0], B_fit_left2[0]])
    
    mean_slope = np.mean([abs(T_fit_left[0]), abs(B_fit_left2[0])], axis = 0)
    slope3_mean.append(mean_slope) #left side
    
    slope4.append([T_fit_right[0], B_fit_right2[0]])
    
    mean_slope = np.mean([abs(T_fit_right[0]), abs(B_fit_right2[0])], axis = 0)
    slope4_mean.append(mean_slope) #right side
    
    xhigh_left = dX[230:360]
    yhigh_left = dTemp[230:360]
    
    # Calculate coefficients
    left_stack = np.vstack([xhigh_left, np.ones(len(xhigh_left))]).T
    m, c = np.linalg.lstsq(left_stack, yhigh_left, rcond=None)[0]
    high_slope = m
    high_intercept = c
    
    xlow_left = dX[10:130]
    ylow_left = dTemp[10:130]
    
    # Calculate coefficients
    left_stack = np.vstack([xlow_left, np.ones(len(xlow_left))]).T
    m, c = np.linalg.lstsq(left_stack, ylow_left, rcond=None)[0]
    low_slope = m
    low_intercept = c
    
    temp_diff = high_intercept - low_intercept
    
    y_high_4 = (high_slope * dX[200]) + high_intercept
    y_low_4 = (low_slope * len(xlow_left/4.05)) + low_intercept
    temp_diff_4 = y_high_4 - y_low_4
    temp_D_200Si.append(temp_diff_4)
    temp_D_200Si_both.append(y_high_4)
    temp_D_200Si_both.append(y_low_4)
    
    x_axis_temp_loc = (1/4.05)*max(dX)*0.1
    values_tempD = np.linspace(y_low_4, y_high_4,10)
    count_tempD_height = len(values_tempD)

    mid_temp = sorted(dTemp)[len(dTemp) // 2]
    mid_temp_200Si.append(mid_temp)
    
    average_temp = np.mean(dTemp)
    average_temp_200Si.append(average_temp)
    
    boundary_high = dTemp[excluded_points1]
    boundary_high_200Si.append(boundary_high)
 
    boundary_low = dTemp[excluded_points1-27]
    boundary_low_200Si.append(boundary_low)
    
    
Slope1 = np.array(slope1)
Slope_mean1 = np.array(slope1_mean)

Slope2 = np.array(slope2)
Slope_mean2 = np.array(slope2_mean)

Slope_mean3 = np.array(slope3_mean)
Slope_mean4 = np.array(slope4_mean)

Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0)]) #individual

Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0]]) #individual

# Calculate the heat flux and the thermal conductivity
eV2J  = 1.6e-19;
Ang2m =   1e-10;
ps2s  =   1e-12;
conversion = (eV2J/(Ang2m*ps2s))

s_values = []
LR_values_200 = []
Thermals_geHot_200 = []
hf_values = []
heat_fluxSi200Si_values = []
HF_constant_left_200Si = []
HF_constant_right_200Si = []

dt = 0.0005 #picoseconds

for simNo in range(0, len(Thermo_Data)):
    
    Steps = Thermo_Data[simNo,-1,0]
    Time = Steps*dt # picoseconds
    
    dx = Thermo_Data[simNo,-1,9]
    dy = Thermo_Data[simNo,-1,10]
    dz = Thermo_Data[simNo,-1,11]
    Area = dy*dz
    
    #DeltaE = Thermo_Data[simNo,-1,-2] # E_pair
    #TotalE = Thermo_Data[simNo,-1,-5] # Total energy
    KinE = Thermo_Data[simNo,-1,12] # in log_lammps, f_3 
    
    Thermal_left2 = ((KinE)/(2*Time*Area*Slope2[simNo,0]))*conversion #top left
    Thermal_right2 = ((KinE)/(2*Time*Area*Slope2[simNo,1]))*conversion #top right
    
    Thermal_left1 = ((KinE)/(2*Time*Area*Slope1[simNo,0]))*conversion #bottom left
    Thermal_right1 = ((KinE)/(2*Time*Area*Slope1[simNo,1]))*conversion #bottom right
    
    thermal2 = np.concatenate((Thermal_left1, Thermal_left2), axis = None)
    thermal1 = np.concatenate((Thermal_right1, Thermal_right2), axis = None)
    
    thermal2_ave = ((KinE)/(2*Time*Area*Slope_mean2[simNo]))*conversion #top
    thermal1_ave = ((KinE)/(2*Time*Area*Slope_mean1[simNo]))*conversion #bottom
    
    thermal3_ave = ((KinE)/(2*Time*Area*Slope_mean3[simNo]))*conversion #left
    thermal4_ave = ((KinE)/(2*Time*Area*Slope_mean4[simNo]))*conversion #right
    
    #thermal_ave = ((KinE)/(2*Time*Area*Slope_ave[simNo]))*conversion
    thermal_ave = (thermal2_ave+thermal1_ave)/2 #top and bottom
    thermal_ave_B = (thermal2_ave+thermal1_ave)/2 #left and right
    
    getcontext().prec = 20
    std_top = Decimal(np.std([Thermal_left2, Thermal_right2], dtype=np.float64))
    SE_left = std_top/Decimal(math.sqrt(2))
   
    std_bottom = Decimal(np.std([Thermal_left1, Thermal_right1], dtype=np.float64))
    SE_right = std_bottom/Decimal(math.sqrt(2))
    
    SE = math.sqrt(((SE_left/2)**2) + ((SE_right/2)**2))
    
    Therm_val = [Thermal_left1,Thermal_left2,Thermal_right1,Thermal_right2]
    Ttherm = ['%.20f' % elem for elem in Therm_val]
    Thermals_geHot_200.append(Ttherm)  
    
    TC_values = [thermal2_ave, thermal1_ave, thermal_ave, SE, thermal3_ave, thermal4_ave]
    val = ['%.20f' % elem for elem in TC_values]
    s_values.append(val)
        
    LR200 = [Thermal_left1, Thermal_left2, Thermal_right2, Thermal_right1]
    
    value = ['%.20f' % elem for elem in LR200]
    LR_values_200.append(value)
    
    flux_top_left = (Thermal_left2*(Slope2[simNo,0]*1e10)*(Area*1e-20))
    flux_top_right = (Thermal_right2*(Slope2[simNo,1]*1e10)*(Area*1e-20))
    flux_bot_left = (Thermal_left1*(Slope2[simNo,0]*1e10)*(Area*1e-20))
    flux_bot_right = (Thermal_right1*(Slope2[simNo,1]*1e10)*(Area*1e-20))
    
    fluxes = [flux_top_left,flux_top_right,flux_bot_left,flux_bot_right]
    heat_val = ['%.20f' % elem for elem in fluxes]
    
    heat_fluxSi200Si_values.append(heat_val)
    
   
    hf_values.append(heat_val)
    sum_hf_values =  sum(fluxes)
    SUM_HF_200Si = []
    SUM_HF_200Si.append(sum_hf_values)
    
    HF_constant_left = (Slope2[simNo,0]*1e10)*(Area*1e-20)
    HF_constant_left_200Si.append(HF_constant_left)

    HF_constant_right = (Slope1[simNo,0]*1e10)*(Area*1e-20)
    HF_constant_right_200Si.append(HF_constant_right)
    

heat_flux_top_left200Si = [];heat_flux_top_right200Si = [];heat_flux_bot_left200Si = [];heat_flux_bot_right200Si = [];
for h in hf_values:
    heat_flux_top_left200Si.append(np.array(float(h[0]))) #flux_top_left
    heat_flux_top_right200Si.append(np.array(float(h[1]))) #flux_top_right
    heat_flux_bot_left200Si.append(np.array(float(h[2]))) #flux_bot_left
    heat_flux_bot_right200Si.append(np.array(float(h[3]))) #flux_bot_right


therm_averageSi200 = []
for item in s_values:
    therm_averageSi200.append(item[2])

therm_averageSi200 = []
for item in s_values:
    therm_averageSi200.append(np.array(float(item[2])))

therm_averageSi200_left = []
for item in s_values:
    therm_averageSi200_left.append(np.array(float(item[4])))


av_silicon_SiGeSi_200 = (float(LR_values_200[0][0]) + float(LR_values_200[0][3]) +
               float(LR_values_200[1][0]) + float(LR_values_200[1][3]) +
               float(LR_values_200[2][0]) + float(LR_values_200[2][3]) +
               float(LR_values_200[3][0]) + float(LR_values_200[3][3])) / 8

av_silicon_SiGeSi_200_values = [float(LR_values_200[0][0]), float(LR_values_200[0][3]),
               float(LR_values_200[1][0]), float(LR_values_200[1][3]),
               float(LR_values_200[2][0]), float(LR_values_200[2][3]),
               float(LR_values_200[3][0]), float(LR_values_200[3][3])]


av_germanium_SiGeSi_200 = (float(LR_values_200[0][1]) + float(LR_values_200[0][2]) +
               float(LR_values_200[1][1]) + float(LR_values_200[1][2]) +
               float(LR_values_200[2][1]) + float(LR_values_200[2][2]) +
               float(LR_values_200[3][1]) + float(LR_values_200[3][2])) / 8

av_germanium_SiGeSi_200_values = [float(LR_values_200[0][1]), float(LR_values_200[0][2]),
               float(LR_values_200[1][1]), float(LR_values_200[1][2]),
               float(LR_values_200[2][1]), float(LR_values_200[2][2]),
               float(LR_values_200[3][1]), float(LR_values_200[3][2])]
 
getcontext().prec = 20
std_silicon_SiGeSi_200 = Decimal(np.std([float(LR_values_200[0][0]),float(LR_values_200[0][3]),
               float(LR_values_200[1][0]),float(LR_values_200[1][3]),
               float(LR_values_200[2][0]),float(LR_values_200[2][3]),
               float(LR_values_200[3][0]),float(LR_values_200[3][3])], dtype=np.float64))

SE_silicon_SiGeSi_200 = std_silicon_SiGeSi_200/Decimal(math.sqrt(8))

getcontext().prec = 20
std_germanium_SiGeSi_200 = Decimal(np.std([float(LR_values_200[0][1]),float(LR_values_200[0][2]),
               float(LR_values_200[1][1]),float(LR_values_200[1][2]),
               float(LR_values_200[2][1]),float(LR_values_200[2][2]),
               float(LR_values_200[3][1]),float(LR_values_200[3][2])], dtype=np.float64))
                      
SE_germanium_SiGeSi_200 = std_germanium_SiGeSi_200/Decimal(math.sqrt(8))

sil = np.array([float(LR_values_200[0][0]),float(LR_values_200[0][3]),
               float(LR_values_200[1][0]),float(LR_values_200[1][3]),
               float(LR_values_200[2][0]),float(LR_values_200[2][3]),
               float(LR_values_200[3][0]),float(LR_values_200[3][3])])
ger = np.array([float(LR_values_200[0][1]),float(LR_values_200[0][2]),
               float(LR_values_200[1][1]),float(LR_values_200[1][2]),
               float(LR_values_200[2][1]),float(LR_values_200[2][2]),
               float(LR_values_200[3][1]),float(LR_values_200[3][2])])


sil_SiGeSi_200_50 = np.array([sil[0],sil[1]])
sil_SiGeSi_200_125 = np.array([sil[2],sil[3]])
sil_SiGeSi_200_250 = np.array([sil[4],sil[5]])
sil_SiGeSi_200_271 = np.array([sil[6],sil[7]])

ger_SiGeSi_200_50 = np.array([ger[0],ger[1]])
ger_SiGeSi_200_125 = np.array([ger[2],ger[3]])
ger_SiGeSi_200_250 = np.array([ger[4],ger[5]])
ger_SiGeSi_200_271 = np.array([ger[6],ger[7]])



In [ ]:
#400

In [ ]:
#600

In [ ]:
#800

In [ ]:
#1000

In [ ]:
#200
## Plot all four simulations 

filename = ['./profile_geE200r50','profile_geE200r125', 'profile_geE200r250', 'profile_geE200r271']

MP_Profile = []; MP_Ave = []; MP_LastN = []; 

for simNo in range(0,len(filename)):
    subprocess.call(r"perl -p -e 's/^[\ \t]+|[\ \t]+$//g' %s > %s_new" % (filename[simNo], filename[simNo]), shell=True) # remove trailling white spaces
    subprocess.call("mv %s_new %s" % (filename[simNo], filename[simNo]), shell=True)
    subprocess.call("tr -s ' ' < %s > %s_tmp " % (filename[simNo], filename[simNo]), shell=True) #squeeze spaces
    subprocess.call("mv %s_tmp %s" % (filename[simNo], filename[simNo]), shell=True)

    profile = []; N = 800;
    with open(filename[simNo], 'r') as f:
        while True:
            lines = list(itertools.islice(f, 1, N+1))
            for line in lines:
                profile.append(line.split(' '))
            if not lines: 
                break
                
    #lastN = profile[-N:] 
    lastAve = profile[-3200:]

    MP_Profile.append(profile)
    #MP_LastN.append(lastN)
    MP_Ave.append(lastAve)
    
MP_Profile = np.array(MP_Profile, dtype=float) # for plotting 3D graphs
#MP_LastN = np.array(MP_LastN, dtype=float) # for calculating the thermal conductivity
MP_Ave = np.array(MP_Ave, dtype=float) # for averaging the data to calculate the thermal conductivity


Thermo_Data = np.array([np.loadtxt('./Thermo_geE200r50'), np.loadtxt('Thermo_geE200r125'), np.loadtxt('Thermo_geE200r250'), np.loadtxt('Thermo_geE200r271')], dtype=object) #, np.loadtxt('Thermo_geE200r125_Original_2')])
                                                                      

labels = ['GeSiGe200r50','GeSiGe200r125', 'GeSiGe200r250', 'GeSiGe200r271'] 

for simNo in range(0, len(MP_Profile)):
    Coordinates = MP_Profile[simNo,:,1]*(Thermo_Data[simNo,-1,9])
    Temperature = MP_Profile[simNo,:,3]

# Load the data that will be averaged to calculate the slopes 
Temperature = []; Coordinate = []; dX_Thermo = [];

for simNo in range(0, len(filename)):
    
    dX_ = Thermo_Data[simNo,-1,9]
    dX_Thermo.append(dX_)
    
    for i in range(0,800):
        Temperature_ = np.mean([MP_Ave[simNo,i,3],MP_Ave[simNo,i+800,3],MP_Ave[simNo,i+1600,3],MP_Ave[simNo,i+2400,3]])   
        Temperature.append(Temperature_)
        Coordinate_ = np.mean([MP_Ave[simNo,i,1],MP_Ave[simNo,i+800,1],MP_Ave[simNo,i+1600,1],MP_Ave[simNo,i+2400,1]])*dX_
        Coordinate.append(Coordinate_)
    
Temperature = np.array([Temperature[0:800],Temperature[800:1600], Temperature[1600:2400], Temperature[2400:3200], 
                        ], dtype=float)
Coordinate = np.array([Coordinate[0:800],Coordinate[800:1600], Coordinate[1600:2400], Coordinate[2400:3200], 
                      ], dtype=float)


excluded_points1 = 230 # excluded points from the beginning
excluded_points2 = 40 # excluded points from the end
excluded_points3 = 120
excluded_points4 = 230 # from the end of the mp.data file

slope2_mean = []; slope2 = []; slope1_mean = []; slope1 = [];
slope3_mean = []; slope3 = []; slope4_mean = []; slope4 = [];
slope2_mean = []; slope2 = []; slope1_mean = []; slope1 = [];temp_D_200Ge_both = []; temp_D_200Ge = [];mid_temp_200Ge = []; average_temp_200Ge = [];
boundary_low_200Ge = []; boundary_high_200Ge = [];

for simNo in range(0,len(filename)):
    
    dTemp = Temperature[simNo,:]
    X = dX_
    dX = Coordinate[simNo,:] #*X # normalized distance
    
    T_fit_left = np.polyfit(dX[excluded_points1:400-excluded_points2], dTemp[excluded_points1:400-excluded_points2], 1)
    T_fit_right = -np.polyfit(dX[400+excluded_points2:-excluded_points1], dTemp[400+excluded_points2:-excluded_points1], 1)
    
    slope2.append([T_fit_left[0], T_fit_right[0]])
    
    mean_slope = np.mean([abs(T_fit_left[0]), abs(T_fit_right[0])], axis = 0)
    slope2_mean.append(mean_slope)
    
    B_fit_left2 = np.polyfit(dX[excluded_points3:400-excluded_points4], dTemp[excluded_points3:400-excluded_points4], 1)
    B_fit_right2 = -np.polyfit(dX[400+excluded_points4:-excluded_points3], dTemp[400+excluded_points4:-excluded_points3], 1)
    
    slope1.append([B_fit_left2[0], B_fit_right2[0]])
    
    mean_slope = np.mean([abs(B_fit_left2[0]), abs(B_fit_right2[0])], axis = 0)
    slope1_mean.append(mean_slope)
    
    slope3.append([T_fit_left[0], B_fit_left2[0]])
    
    mean_slope = np.mean([abs(T_fit_left[0]), abs(B_fit_left2[0])], axis = 0)
    slope3_mean.append(mean_slope) #left side
    
    slope4.append([T_fit_right[0], B_fit_right2[0]])
    
    mean_slope = np.mean([abs(T_fit_right[0]), abs(B_fit_right2[0])], axis = 0)
    slope4_mean.append(mean_slope) #right side
    
    xhigh_left = dX[230:390]
    yhigh_left = dTemp[230:390]
    
    # Calculate coefficients
    left_stack = np.vstack([xhigh_left, np.ones(len(xhigh_left))]).T
    m, c = np.linalg.lstsq(left_stack, yhigh_left, rcond=None)[0]
    high_slope = m
    high_intercept = c
    
    xlow_left = dX[10:170]
    ylow_left = dTemp[10:170]
    
    # Calculate coefficients
    left_stack = np.vstack([xlow_left, np.ones(len(xlow_left))]).T
    m, c = np.linalg.lstsq(left_stack, ylow_left, rcond=None)[0]
    low_slope = m
    low_intercept = c
    
    temp_diff = high_intercept - low_intercept
      
    y_high_4 = (high_slope * dX[205]) + high_intercept
    y_low_4 = (low_slope * len(xlow_left/3.9)) + low_intercept
    temp_diff_4 = y_high_4 - y_low_4
    temp_D_200Ge.append(temp_diff_4)
    temp_D_200Ge_both.append(y_high_4)
    temp_D_200Ge_both.append(y_low_4)

    x_axis_temp_loc = (1/3.9)*max(dX)*0.1
    values_tempD = np.linspace(y_low_4, y_high_4,10)
    count_tempD_height = len(values_tempD)

    mid_temp = sorted(dTemp)[len(dTemp) // 2]
    mid_temp_200Ge.append(mid_temp)
    
    average_temp = np.mean(dTemp)
    average_temp_200Ge.append(average_temp)

    boundary_high = dTemp[excluded_points1]
    boundary_high_200Ge.append(boundary_high)
 
    boundary_low = dTemp[excluded_points1-27]
    boundary_low_200Ge.append(boundary_low)

Slope1 = np.array(slope1)
Slope_mean1 = np.array(slope1_mean)

Slope2 = np.array(slope2)
Slope_mean2 = np.array(slope2_mean)

Slope_mean3 = np.array(slope3_mean)
Slope_mean4 = np.array(slope4_mean)

Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0)]) #individual

Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0]]) #individual

# Calculate the heat flux and the thermal conductivity
eV2J  = 1.6e-19;
Ang2m =   1e-10;
ps2s  =   1e-12;
conversion = (eV2J/(Ang2m*ps2s))

s_values = []
LR_values_200 = []
Thermals_siHot_200 = []
hf_values = [];heat_flux200Ge_values = []
HF_constant_left_200Ge = []
HF_constant_right_200Ge = []

dt = 0.0005 # picoseconds

for simNo in range(0, len(Thermo_Data)):
    
    Steps = Thermo_Data[simNo,-1,0]
    Time = Steps*dt # picoseconds
    
    dx = Thermo_Data[simNo,-1,9]
    dy = Thermo_Data[simNo,-1,10]
    dz = Thermo_Data[simNo,-1,11]
    Area = dy*dz
    
    #DeltaE = Thermo_Data[simNo,-1,-2] # E_pair
    #TotalE = Thermo_Data[simNo,-1,-5] # Total energy
    KinE = Thermo_Data[simNo,-1,12] # in log_lammps, f_3 
    
    Thermal_left2 = ((KinE)/(2*Time*Area*Slope2[simNo,0]))*conversion #top left
    Thermal_right2 = ((KinE)/(2*Time*Area*Slope2[simNo,1]))*conversion #top right
    
    Thermal_left1 = ((KinE)/(2*Time*Area*Slope1[simNo,0]))*conversion #bottom left
    Thermal_right1 = ((KinE)/(2*Time*Area*Slope1[simNo,1]))*conversion #bottom right
    
    thermal2 = np.concatenate((Thermal_left1, Thermal_left2), axis = None)
    thermal1 = np.concatenate((Thermal_right1, Thermal_right2), axis = None)
    
    thermal2_ave = ((KinE)/(2*Time*Area*Slope_mean2[simNo]))*conversion #top
    thermal1_ave = ((KinE)/(2*Time*Area*Slope_mean1[simNo]))*conversion #bottom
    
    thermal3_ave = ((KinE)/(2*Time*Area*Slope_mean3[simNo]))*conversion #left
    thermal4_ave = ((KinE)/(2*Time*Area*Slope_mean4[simNo]))*conversion #right

    #thermal_ave = ((KinE)/(2*Time*Area*Slope_ave[simNo]))*conversion
    thermal_ave = (thermal2_ave+thermal1_ave)/2
    
    getcontext().prec = 20
    std_top = Decimal(np.std([Thermal_left2, Thermal_right2], dtype=np.float64))
    SE_left = std_top/Decimal(math.sqrt(2))
   
    std_bottom = Decimal(np.std([Thermal_left1, Thermal_right1], dtype=np.float64))
    SE_right = std_bottom/Decimal(math.sqrt(2))
    
    SE = math.sqrt(((SE_left/2)**2) + ((SE_right/2)**2))
    
    Therm_val = [Thermal_left1,Thermal_left2,Thermal_right1,Thermal_right2]
    Ttherm = ['%.20f' % elem for elem in Therm_val]
    
    Thermals_siHot_200.append(Ttherm)
    
    TC_values = [thermal2_ave, thermal1_ave, thermal_ave, SE, thermal3_ave, thermal4_ave]
    val = ['%.20f' % elem for elem in TC_values]
    s_values.append(val)
    
    
    LR200 = [Thermal_left1, Thermal_left2, Thermal_right2, Thermal_right1]
    value = ['%.20f' % elem for elem in LR200]
    
    LR_values_200.append(value)
    
    df = Slope2[simNo,0]*1e10    

    flux_top_left = (Thermal_left2*(Slope2[simNo,0]*1e10)*(Area*1e-20))
    flux_top_right = (Thermal_right2*(Slope2[simNo,1]*1e10)*(Area*1e-20))
    flux_bot_left = (Thermal_left1*(Slope2[simNo,0]*1e10)*(Area*1e-20))
    flux_bot_right = (Thermal_right1*(Slope2[simNo,1]*1e10)*(Area*1e-20))
    
    fluxes = [flux_top_left,flux_top_right,flux_bot_left,flux_bot_right]
    heat_val = ['%.20f' % elem for elem in fluxes]
    
    heat_flux200Ge_values.append(heat_val)

   
    hf_values.append(heat_val)
    sum_hf_values =  sum(fluxes)
    SUM_HF_200Ge = []
    SUM_HF_200Ge.append(sum_hf_values)
    
    HF_constant_left = (Slope2[simNo,0]*1e10)*(Area*1e-20)
    HF_constant_left_200Ge.append(HF_constant_left)

    HF_constant_right = (Slope1[simNo,0]*1e10)*(Area*1e-20)
    HF_constant_right_200Ge.append(HF_constant_right)
    
heat_flux_top_left200Ge = [];heat_flux_top_right200Ge = [];heat_flux_bot_left200Ge = [];heat_flux_bot_right200Ge = [];
for h in hf_values:
    heat_flux_top_left200Ge.append(np.array(float(h[0]))) #flux_top_left
    #heat_flux_top_right200Ge.append(np.array(float(h[1]))) flux_top_right
    #heat_flux_bot_left200Ge.append(np.array(float(h[2]))) flux_bot_left
    #heat_flux_bot_right200Ge.append(np.array(float(h[3]))) flux_bot_right



therm_averageGe200 = []
for item in s_values:
    therm_averageGe200.append(item[2])

therm_averageGe200 = []
for item in s_values:
    therm_averageGe200.append(np.array(float(item[2])))


av_germanium_GeSiGe_200 = (float(LR_values_200[0][0]) + float(LR_values_200[0][3]) +
               float(LR_values_200[1][0]) + float(LR_values_200[1][3]) +
               float(LR_values_200[2][0]) + float(LR_values_200[2][3]) +
               float(LR_values_200[3][0]) + float(LR_values_200[3][3])) / 8

av_germanium_GeSiGe_200_values = [float(LR_values_200[0][0]), float(LR_values_200[0][3]),
               float(LR_values_200[1][0]), float(LR_values_200[1][3]),
               float(LR_values_200[2][0]), float(LR_values_200[2][3]),
               float(LR_values_200[3][0]), float(LR_values_200[3][3])]


av_silicon_GeSiGe_200 = (float(LR_values_200[0][1]) + float(LR_values_200[0][2]) +
               float(LR_values_200[1][1]) + float(LR_values_200[1][2]) +
               float(LR_values_200[2][1]) + float(LR_values_200[2][2]) +
               float(LR_values_200[3][1]) + float(LR_values_200[3][2])) / 8

av_silicon_GeSiGe_200_values = [float(LR_values_200[0][1]), float(LR_values_200[0][2]),
               float(LR_values_200[1][1]), float(LR_values_200[1][2]),
               float(LR_values_200[2][1]), float(LR_values_200[2][2]),
               float(LR_values_200[3][1]), float(LR_values_200[3][2])]

getcontext().prec = 20
std_germanium_GeSiGe_200 = Decimal(np.std([float(LR_values_200[0][0]),float(LR_values_200[0][3]),
               float(LR_values_200[1][0]),float(LR_values_200[1][3]),
               float(LR_values_200[2][0]),float(LR_values_200[2][3]),
               float(LR_values_200[3][0]),float(LR_values_200[3][3])], dtype=np.float64))
                      
SE_germanium_GeSiGe_200 = std_germanium_GeSiGe_200/Decimal(math.sqrt(8))

getcontext().prec = 20
std_silicon_GeSiGe_200 = Decimal(np.std([float(LR_values_200[0][1]),float(LR_values_200[0][2]),
               float(LR_values_200[1][1]),float(LR_values_200[1][2]),
               float(LR_values_200[2][1]),float(LR_values_200[2][2]),
               float(LR_values_200[3][1]),float(LR_values_200[3][2])], dtype=np.float64))
                      
SE_silicon_GeSiGe_200 = std_silicon_GeSiGe_200/Decimal(math.sqrt(8))

ger = np.array([float(LR_values_200[0][0]),float(LR_values_200[0][3]),
               float(LR_values_200[1][0]),float(LR_values_200[1][3]),
               float(LR_values_200[2][0]),float(LR_values_200[2][3]),
               float(LR_values_200[3][0]),float(LR_values_200[3][3])])
sil = np.array([float(LR_values_200[0][1]),float(LR_values_200[0][2]),
               float(LR_values_200[1][1]),float(LR_values_200[1][2]),
               float(LR_values_200[2][1]),float(LR_values_200[2][2]),
               float(LR_values_200[3][1]),float(LR_values_200[3][2])])

ger_GeSiGe_200_50 = np.array([ger[0],ger[1]])
ger_GeSiGe_200_125 = np.array([ger[2],ger[3]])
ger_GeSiGe_200_250 = np.array([ger[4],ger[5]])
ger_GeSiGe_200_271 = np.array([ger[6],ger[7]])

sil_GeSiGe_200_50 = np.array([sil[0],sil[1]])
sil_GeSiGe_200_125 = np.array([sil[2],sil[3]])
sil_GeSiGe_200_250 = np.array([sil[4],sil[5]])
sil_GeSiGe_200_271 = np.array([sil[6],sil[7]])

In [ ]:
#R_int = delta T/J
#J = delta KE/Ac.delta t
#delta t = 0.5 fs
A_t_nm = 0.1
A_t_m = 1e-10


radius_sq = ((54.3*A_t_m)/2)**2
print(radius_sq)
print(((54.3*A_t_m)/2))
print(54.3*A_t_m)

cross_section = np.pi*radius_sq

d_t = 3e-9

Ac_dt = 2* d_t

In [ ]:
#Si
#200
R_int_s200r50 = temp_D_200Si[0]/np.sum((float(heat_fluxSi200Si_values[0][0]),float(heat_fluxSi200Si_values[0][2])), dtype=np.float32)

R_int_s200r125 = temp_D_200Si[1]/np.sum((float(heat_fluxSi200Si_values[1][0]),float(heat_fluxSi200Si_values[1][2])), dtype=np.float32)

R_int_s200r250 = temp_D_200Si[2]/np.sum((float(heat_fluxSi200Si_values[2][0]),float(heat_fluxSi200Si_values[2][2])), dtype=np.float32)

R_int_s200r271 = temp_D_200Si[3]/np.sum((float(heat_fluxSi200Si_values[3][0]),float(heat_fluxSi200Si_values[3][2])), dtype=np.float32)



In [ ]:
#Ge
#200
#print(heat_flux200Ge_values[0][2])
#print(temp_D_200Ge[0])
R_int_g200r50 = temp_D_200Ge[0]/np.sum((float(heat_flux200Ge_values[0][0]),float(heat_flux200Ge_values[0][2])), dtype=np.float32)

R_int_g200r125 = temp_D_200Ge[1]/np.sum((float(heat_flux200Ge_values[1][0]),float(heat_flux200Ge_values[1][2])), dtype=np.float32)

R_int_g200r250 = temp_D_200Ge[2]/np.sum((float(heat_flux200Ge_values[2][0]),float(heat_flux200Ge_values[2][2])), dtype=np.float32)

R_int_g200r271 = temp_D_200Ge[3]/np.sum((float(heat_flux200Ge_values[3][0]),float(heat_flux200Ge_values[3][2])), dtype=np.float32)

In [ ]:
#Standard errors
#200
#SE thermal conductivity
SE200geHot_r50_left = np.std([float(Thermals_geHot_200[0][0]), float(Thermals_geHot_200[0][1])], dtype=np.float64)
SE200geHot_r50_left = SE200geHot_r50_left/math.sqrt(2)
SE200geHot_r50_right = np.std([float(Thermals_geHot_200[0][2]), float(Thermals_geHot_200[0][3])], dtype=np.float64)
SE200geHot_r50_right = SE200geHot_r50_right/math.sqrt(2)
standard_error200geHot_r50 = math.sqrt(((SE200geHot_r50_left/2)**2) + ((SE200geHot_r50_right/2)**2))
#print('Standard error of size 200-r50:',standard_error200geHot_r50)

SE200geHot_r125_left = np.std([float(Thermals_geHot_200[1][0]), float(Thermals_geHot_200[1][1])], dtype=np.float64)
SE200geHot_r125_left = SE200geHot_r125_left/math.sqrt(2)
SE200geHot_r125_right = np.std([float(Thermals_geHot_200[1][2]), float(Thermals_geHot_200[1][3])], dtype=np.float64)
SE200geHot_r125_right = SE200geHot_r125_right/math.sqrt(2)
standard_error200geHot_r125 = math.sqrt(((SE200geHot_r125_left/2)**2) + ((SE200geHot_r125_right/2)**2))
#print('Standard error of size 200-r125:',standard_error200geHot_r125)

SE200geHot_r250_left = np.std([float(Thermals_geHot_200[2][0]), float(Thermals_geHot_200[2][1])], dtype=np.float64)
SE200geHot_r250_left = SE200geHot_r250_left/math.sqrt(2)
SE200geHot_r250_right = np.std([float(Thermals_geHot_200[2][2]), float(Thermals_geHot_200[2][3])], dtype=np.float64)
SE200geHot_r250_right = SE200geHot_r250_right/math.sqrt(2)
standard_error200geHot_r250 = math.sqrt(((SE200geHot_r250_left/2)**2) + ((SE200geHot_r250_right/2)**2))
#print('Standard error of size 200-r250:',standard_error200geHot_r250)

SE200geHot_r271_left = np.std([float(Thermals_geHot_200[3][0]), float(Thermals_geHot_200[3][1])], dtype=np.float64)
SE200geHot_r271_left = SE200geHot_r271_left/math.sqrt(2)
SE200geHot_r271_right = np.std([float(Thermals_geHot_200[3][2]), float(Thermals_geHot_200[3][3])], dtype=np.float64)
SE200geHot_r271_right = SE200geHot_r271_right/math.sqrt(2)
standard_error200geHot_r271 = math.sqrt(((SE200geHot_r271_left/2)**2) + ((SE200geHot_r271_right/2)**2))
print('Standard error of tc 200-r271:',standard_error200geHot_r271)

#SE for delta T
SE200geHot_r50_dt = np.std([float(temp_D_200Si_both[0]), float(temp_D_200Si_both[1])], dtype=np.float64)
standard_error200geHot_r50_dt = SE200geHot_r50_dt/math.sqrt(2)
#print('Standard error of size 200-r50:',standard_error200geHot_r50_dt)

SE200geHot_r125_dt = np.std([float(temp_D_200Si_both[2]), float(temp_D_200Si_both[3])], dtype=np.float64)
standard_error200geHot_r125_dt = SE200geHot_r125_dt/math.sqrt(2)
#print('Standard error of size 200-r125:',standard_error200geHot_r125_dt)

SE200geHot_r250_dt = np.std([float(temp_D_200Si_both[4]), float(temp_D_200Si_both[5])], dtype=np.float64)
standard_error200geHot_r250_dt = SE200geHot_r250_dt/math.sqrt(2)
#print('Standard error of size 200-r250:',standard_error200geHot_r250_dt)

SE200geHot_r271_dt = np.std([float(temp_D_200Si_both[0]), float(temp_D_200Si_both[1])], dtype=np.float64)
standard_error200geHot_r271_dt = SE200geHot_r271_dt/math.sqrt(2)
print('Standard error of dt 200-r271:',standard_error200geHot_r271_dt)

#SE for R_int
#Used only the left side to find Rint

#SE Rint1 = Rint1 * sqrt( (SE delta t / delta t)² + (c * delta TC / c * TC)² )
#SE Rint = sqrt( (SE Rint1)² + (SE Rint2)² ...)

SE200geHot_r50_rint = R_int_s200r50 * math.sqrt( (SE200geHot_r50_dt/temp_D_200Si[0])**2 +
                                                ( (HF_constant_left_200Si[0]*SE200geHot_r50_left) / (HF_constant_left_200Si[0]*therm_averageSi200_left[0]) )**2 )
    
SE200geHot_r125_rint = R_int_s200r125 * math.sqrt( (SE200geHot_r125_dt/temp_D_200Si[1])**2 +
                                                ( (HF_constant_left_200Si[1]*SE200geHot_r125_left) / (HF_constant_left_200Si[1]*therm_averageSi200_left[1]) )**2 )
    
SE200geHot_r250_rint = R_int_s200r250 * math.sqrt( (SE200geHot_r250_dt/temp_D_200Si[2])**2 +
                                                ( (HF_constant_left_200Si[2]*SE200geHot_r250_left) / (HF_constant_left_200Si[2]*therm_averageSi200_left[2]) )**2 )
    
SE200geHot_r271_rint = R_int_s200r271 * math.sqrt( (SE200geHot_r271_dt/temp_D_200Si[3])**2 +
                                                ( (HF_constant_left_200Si[3]*SE200geHot_r271_left) / (HF_constant_left_200Si[3]*therm_averageSi200_left[3]) )**2 )
    

In [ ]:
#Standard errors
#200
#SE thermal conductivity
SE200siHot_r50_left = np.std([float(Thermals_siHot_200[0][0]), float(Thermals_siHot_200[0][1])], dtype=np.float64)
SE200siHot_r50_left = SE200siHot_r50_left/math.sqrt(2)
SE200siHot_r50_right = np.std([float(Thermals_siHot_200[0][2]), float(Thermals_siHot_200[0][3])], dtype=np.float64)
SE200siHot_r50_right = SE200siHot_r50_right/math.sqrt(2)
standard_error200siHot_r50 = math.sqrt(((SE200siHot_r50_left/2)**2) + ((SE200siHot_r50_right/2)**2))
#print('Standard error of size 200-r50:',standard_error200siHot_r50)

SE200siHot_r125_left = np.std([float(Thermals_siHot_200[1][0]), float(Thermals_siHot_200[1][1])], dtype=np.float64)
SE200siHot_r125_left = SE200siHot_r125_left/math.sqrt(2)
SE200siHot_r125_right = np.std([float(Thermals_siHot_200[1][2]), float(Thermals_siHot_200[1][3])], dtype=np.float64)
SE200siHot_r125_right = SE200siHot_r125_right/math.sqrt(2)
standard_error200siHot_r125 = math.sqrt(((SE200siHot_r125_left/2)**2) + ((SE200siHot_r125_right/2)**2))
#print('Standard error of size 200-r125:',standard_error200siHot_r125)

SE200siHot_r250_left = np.std([float(Thermals_siHot_200[2][0]), float(Thermals_siHot_200[2][1])], dtype=np.float64)
SE200siHot_r250_left = SE200siHot_r250_left/math.sqrt(2)
SE200siHot_r250_right = np.std([float(Thermals_siHot_200[2][2]), float(Thermals_siHot_200[2][3])], dtype=np.float64)
SE200siHot_r250_right = SE200siHot_r250_right/math.sqrt(2)
standard_error200siHot_r250 = math.sqrt(((SE200siHot_r250_left/2)**2) + ((SE200siHot_r250_right/2)**2))
#print('Standard error of size 200-r250:',standard_error200siHot_r250)

SE200siHot_r271_left = np.std([float(Thermals_siHot_200[3][0]), float(Thermals_siHot_200[3][1])], dtype=np.float64)
SE200siHot_r271_left = SE200siHot_r271_left/math.sqrt(2)
SE200siHot_r271_right = np.std([float(Thermals_siHot_200[3][2]), float(Thermals_siHot_200[3][3])], dtype=np.float64)
SE200siHot_r271_right = SE200siHot_r271_right/math.sqrt(2)
standard_error200siHot_r271 = math.sqrt(((SE200siHot_r271_left/2)**2) + ((SE200siHot_r271_right/2)**2))
print('Standard error of tc 200-r271:',standard_error200siHot_r271)

#SE for delta T
SE200siHot_r50_dt = np.std([float(temp_D_200Ge_both[0]), float(temp_D_200Ge_both[1])], dtype=np.float64)
standard_error200siHot_r50_dt = SE200siHot_r50_dt/math.sqrt(2)
#print('Standard error of size 200-r50:',standard_error200siHot_r50_dt)

SE200siHot_r125_dt = np.std([float(temp_D_200Ge_both[2]), float(temp_D_200Ge_both[3])], dtype=np.float64)
standard_error200siHot_r125_dt = SE200siHot_r125_dt/math.sqrt(2)
#print('Standard error of size 200-r125:',standard_error200siHot_r125_dt)

SE200siHot_r250_dt = np.std([float(temp_D_200Ge_both[4]), float(temp_D_200Ge_both[5])], dtype=np.float64)
standard_error200siHot_r250_dt = SE200siHot_r250_dt/math.sqrt(2)
#print('Standard error of size 200-r250:',standard_error200siHot_r250_dt)

SE200siHot_r271_dt = np.std([float(temp_D_200Ge_both[0]), float(temp_D_200Ge_both[1])], dtype=np.float64)
standard_error200siHot_r271_dt = SE200siHot_r271_dt/math.sqrt(2)
print('Standard error of dt 200-r271:',standard_error200siHot_r271_dt)

#SE for R_int
#Used only the left side to find Rint

#SE Rint1 = Rint1 * sqrt( (SE delta t / delta t)² + (c * delta TC / c * TC)² )
#SE Rint = sqrt( (SE Rint1)² + (SE Rint2)² ...)

SE200siHot_r50_rint = R_int_s200r50 * math.sqrt( (SE200siHot_r50_dt/temp_D_200Si[0])**2 +
                                                ( (HF_constant_left_200Si[0]*SE200siHot_r50_left) / (HF_constant_left_200Si[0]*therm_averageSi200_left[0]) )**2 )
    
SE200siHot_r125_rint = R_int_s200r125 * math.sqrt( (SE200siHot_r125_dt/temp_D_200Si[1])**2 +
                                                ( (HF_constant_left_200Si[1]*SE200siHot_r125_left) / (HF_constant_left_200Si[1]*therm_averageSi200_left[1]) )**2 )
    
SE200siHot_r250_rint = R_int_s200r250 * math.sqrt( (SE200siHot_r250_dt/temp_D_200Si[2])**2 +
                                                ( (HF_constant_left_200Si[2]*SE200siHot_r250_left) / (HF_constant_left_200Si[2]*therm_averageSi200_left[2]) )**2 )
    
SE200siHot_r271_rint = R_int_s200r271 * math.sqrt( (SE200siHot_r271_dt/temp_D_200Si[3])**2 +
                                                ( (HF_constant_left_200Si[3]*SE200siHot_r271_left) / (HF_constant_left_200Si[3]*therm_averageSi200_left[3]) )**2 )
    

In [ ]:
#Average Rint error by color
SE_geHot_r50_rint = (math.sqrt( (SE200geHot_r50_rint)**2 + (SE400geHot_r50_rint)**2 + 
                              (SE600geHot_r50_rint)**2 + (SE800geHot_r50_rint)**2 + (SE1000geHot_r50_rint)**2 ))/1e8

SE_geHot_r125_rint = (math.sqrt( (SE200geHot_r125_rint)**2 + (SE400geHot_r125_rint)**2 + 
                              (SE600geHot_r125_rint)**2 + (SE800geHot_r125_rint)**2 + (SE1000geHot_r125_rint)**2 ))/1e8

SE_geHot_r250_rint = (math.sqrt( (SE200geHot_r250_rint)**2 + (SE400geHot_r250_rint)**2 + 
                              (SE600geHot_r250_rint)**2 + (SE800geHot_r250_rint)**2 + (SE1000geHot_r250_rint)**2 ))/1e8

SE_geHot_r271_rint = (math.sqrt( (SE200geHot_r271_rint)**2 + (SE400geHot_r271_rint)**2 + 
                              (SE600geHot_r271_rint)**2 + (SE800geHot_r271_rint)**2 + (SE1000geHot_r271_rint)**2 ))/1e8

#Average Rint error by size
SE_geHot_200_rint = (math.sqrt( (SE200geHot_r50_rint)**2 + (SE200geHot_r125_rint)**2 +
                              (SE200geHot_r250_rint)**2 + (SE200geHot_r271_rint)**2 ))/1e8
     
SE_geHot_400_rint = (math.sqrt( (SE400geHot_r50_rint)**2 + (SE400geHot_r125_rint)**2 +
                              (SE400geHot_r250_rint)**2 + (SE400geHot_r271_rint)**2 ))/1e8
 
SE_geHot_600_rint = (math.sqrt( (SE600geHot_r50_rint)**2 + (SE600geHot_r125_rint)**2 +
                              (SE600geHot_r250_rint)**2 + (SE600geHot_r271_rint)**2 ))/1e8

SE_geHot_800_rint = (math.sqrt( (SE800geHot_r50_rint)**2 + (SE800geHot_r125_rint)**2 +
                              (SE800geHot_r250_rint)**2 + (SE800geHot_r271_rint)**2 ))/1e8

SE_geHot_1000_rint = (math.sqrt( (SE1000geHot_r50_rint)**2 + (SE1000geHot_r125_rint)**2 +
                              (SE1000geHot_r250_rint)**2 + (SE1000geHot_r271_rint)**2 ))/1e8
   

#Average Rint error by color
SE_siHot_r50_rint = (math.sqrt( (SE200siHot_r50_rint)**2 + (SE400siHot_r50_rint)**2 + 
                              (SE600siHot_r50_rint)**2 + (SE800siHot_r50_rint)**2 + (SE1000siHot_r50_rint)**2 ))/1e8

SE_siHot_r125_rint = (math.sqrt( (SE200siHot_r125_rint)**2 + (SE400siHot_r125_rint)**2 + 
                              (SE600siHot_r125_rint)**2 + (SE800siHot_r125_rint)**2 + (SE1000siHot_r125_rint)**2 ))/1e8

SE_siHot_r250_rint = (math.sqrt( (SE200siHot_r250_rint)**2 + (SE400siHot_r250_rint)**2 + 
                              (SE600siHot_r250_rint)**2 + (SE800siHot_r250_rint)**2 + (SE1000siHot_r250_rint)**2 ))/1e8

SE_siHot_r271_rint = (math.sqrt( (SE200siHot_r271_rint)**2 + (SE400siHot_r271_rint)**2 + 
                              (SE600siHot_r271_rint)**2 + (SE800siHot_r271_rint)**2 + (SE1000siHot_r271_rint)**2 ))/1e8

#Average Rint error by size
SE_siHot_200_rint = (math.sqrt( (SE200siHot_r50_rint)**2 + (SE200siHot_r125_rint)**2 +
                              (SE200siHot_r250_rint)**2 + (SE200siHot_r271_rint)**2 ))/1e8
     
SE_siHot_400_rint = (math.sqrt( (SE400siHot_r50_rint)**2 + (SE400siHot_r125_rint)**2 +
                              (SE400siHot_r250_rint)**2 + (SE400siHot_r271_rint)**2 ))/1e8
 
SE_siHot_600_rint = (math.sqrt( (SE600siHot_r50_rint)**2 + (SE600siHot_r125_rint)**2 +
                              (SE600siHot_r250_rint)**2 + (SE600siHot_r271_rint)**2 ))/1e8

SE_siHot_800_rint = (math.sqrt( (SE800siHot_r50_rint)**2 + (SE800siHot_r125_rint)**2 +
                              (SE800siHot_r250_rint)**2 + (SE800siHot_r271_rint)**2 ))/1e8

SE_siHot_1000_rint = (math.sqrt( (SE1000siHot_r50_rint)**2 + (SE1000siHot_r125_rint)**2 +
                              (SE1000siHot_r250_rint)**2 + (SE1000siHot_r271_rint)**2 ))/1e8


In [ ]:
#SE for top boundary temperatures by color
SE_geHot_r50_boundary_high = np.std([boundary_high_200Si[0],boundary_high_400Si[0],boundary_high_600Si[0],boundary_high_800Si[0],boundary_high_1000Si[0]])
SE_geHot_r50_boundary_high = SE_geHot_r50_boundary_high/math.sqrt(5)

SE_geHot_r125_boundary_high = np.std([boundary_high_200Si[1],boundary_high_400Si[1],boundary_high_600Si[1],boundary_high_800Si[1],boundary_high_1000Si[1]])
SE_geHot_r125_boundary_high = SE_geHot_r125_boundary_high/math.sqrt(5)

SE_geHot_r250_boundary_high = np.std([boundary_high_200Si[2],boundary_high_400Si[2],boundary_high_600Si[2],boundary_high_800Si[2],boundary_high_1000Si[2]])
SE_geHot_r250_boundary_high = SE_geHot_r250_boundary_high/math.sqrt(5)

SE_geHot_r271_boundary_high = np.std([boundary_high_200Si[3],boundary_high_400Si[3],boundary_high_600Si[3],boundary_high_800Si[3],boundary_high_1000Si[3]])
SE_geHot_r271_boundary_high = SE_geHot_r271_boundary_high/math.sqrt(5)


#Average boundary_high error by size
SE_geHot_200_boundary_high = np.std([boundary_high_200Si[0],boundary_high_200Si[1],boundary_high_200Si[2],boundary_high_200Si[3]])
SE_geHot_200_boundary_high = SE_geHot_200_boundary_high/math.sqrt(5)

SE_geHot_400_boundary_high = np.std([boundary_high_400Si[0],boundary_high_400Si[1],boundary_high_400Si[2],boundary_high_400Si[3]])
SE_geHot_400_boundary_high = SE_geHot_400_boundary_high/math.sqrt(5)

SE_geHot_600_boundary_high = np.std([boundary_high_600Si[0],boundary_high_600Si[1],boundary_high_600Si[2],boundary_high_600Si[3]])
SE_geHot_600_boundary_high = SE_geHot_600_boundary_high/math.sqrt(5)

SE_geHot_800_boundary_high = np.std([boundary_high_800Si[0],boundary_high_800Si[1],boundary_high_800Si[2],boundary_high_800Si[3]])
SE_geHot_800_boundary_high = SE_geHot_800_boundary_high/math.sqrt(5)

SE_geHot_1000_boundary_high = np.std([boundary_high_1000Si[0],boundary_high_1000Si[1],boundary_high_1000Si[2],boundary_high_1000Si[3]])
SE_geHot_1000_boundary_high = SE_geHot_1000_boundary_high/math.sqrt(5)

#SE for top boundary temperatures by color
SE_siHot_r50_boundary_high = np.std([boundary_high_200Ge[0],boundary_high_400Ge[0],boundary_high_600Ge[0],boundary_high_800Ge[0],boundary_high_1000Ge[0]])
SE_siHot_r50_boundary_high = SE_siHot_r50_boundary_high/math.sqrt(5)

SE_siHot_r125_boundary_high = np.std([boundary_high_200Ge[1],boundary_high_400Ge[1],boundary_high_600Ge[1],boundary_high_800Ge[1],boundary_high_1000Ge[1]])
SE_siHot_r125_boundary_high = SE_siHot_r125_boundary_high/math.sqrt(5)

SE_siHot_r250_boundary_high = np.std([boundary_high_200Ge[2],boundary_high_400Ge[2],boundary_high_600Ge[2],boundary_high_800Ge[2],boundary_high_1000Ge[2]])
SE_siHot_r250_boundary_high = SE_siHot_r250_boundary_high/math.sqrt(5)

SE_siHot_r271_boundary_high = np.std([boundary_high_200Ge[3],boundary_high_400Ge[3],boundary_high_600Ge[3],boundary_high_800Ge[3],boundary_high_1000Ge[3]])
SE_siHot_r271_boundary_high = SE_siHot_r271_boundary_high/math.sqrt(5)


#Average boundary_high error by size
SE_siHot_200_boundary_high = np.std([boundary_high_200Ge[0],boundary_high_200Ge[1],boundary_high_200Ge[2],boundary_high_200Ge[3]])
SE_siHot_200_boundary_high = SE_siHot_200_boundary_high/math.sqrt(5)

SE_siHot_400_boundary_high = np.std([boundary_high_400Ge[0],boundary_high_400Ge[1],boundary_high_400Ge[2],boundary_high_400Ge[3]])
SE_siHot_400_boundary_high = SE_siHot_400_boundary_high/math.sqrt(5)

SE_siHot_600_boundary_high = np.std([boundary_high_600Ge[0],boundary_high_600Ge[1],boundary_high_600Ge[2],boundary_high_600Ge[3]])
SE_siHot_600_boundary_high = SE_siHot_600_boundary_high/math.sqrt(5)

SE_siHot_800_boundary_high = np.std([boundary_high_800Ge[0],boundary_high_800Ge[1],boundary_high_800Ge[2],boundary_high_800Ge[3]])
SE_siHot_800_boundary_high = SE_siHot_800_boundary_high/math.sqrt(5)

SE_siHot_1000_boundary_high = np.std([boundary_high_1000Ge[0],boundary_high_1000Ge[1],boundary_high_1000Ge[2],boundary_high_1000Ge[3]])
SE_siHot_1000_boundary_high = SE_siHot_1000_boundary_high/math.sqrt(5)

In [ ]:
#SE for bottom boundary temperatures by color
SE_geHot_r50_boundary_low = np.std([boundary_low_200Si[0],boundary_low_400Si[0],boundary_low_600Si[0],boundary_low_800Si[0],boundary_low_1000Si[0]])
SE_geHot_r50_boundary_low = SE_geHot_r50_boundary_low/math.sqrt(5)

SE_geHot_r125_boundary_low = np.std([boundary_low_200Si[1],boundary_low_400Si[1],boundary_low_600Si[1],boundary_low_800Si[1],boundary_low_1000Si[1]])
SE_geHot_r125_boundary_low = SE_geHot_r125_boundary_low/math.sqrt(5)

SE_geHot_r250_boundary_low = np.std([boundary_low_200Si[2],boundary_low_400Si[2],boundary_low_600Si[2],boundary_low_800Si[2],boundary_low_1000Si[2]])
SE_geHot_r250_boundary_low = SE_geHot_r250_boundary_low/math.sqrt(5)

SE_geHot_r271_boundary_low = np.std([boundary_low_200Si[3],boundary_low_400Si[3],boundary_low_600Si[3],boundary_low_800Si[3],boundary_low_1000Si[3]])
SE_geHot_r271_boundary_low = SE_geHot_r271_boundary_low/math.sqrt(5)


#Average boundary_low error by size
SE_geHot_200_boundary_low = np.std([boundary_low_200Si[0],boundary_low_200Si[1],boundary_low_200Si[2],boundary_low_200Si[3]])
SE_geHot_200_boundary_low = SE_geHot_200_boundary_low/math.sqrt(5)

SE_geHot_400_boundary_low = np.std([boundary_low_400Si[0],boundary_low_400Si[1],boundary_low_400Si[2],boundary_low_400Si[3]])
SE_geHot_400_boundary_low = SE_geHot_400_boundary_low/math.sqrt(5)

SE_geHot_600_boundary_low = np.std([boundary_low_600Si[0],boundary_low_600Si[1],boundary_low_600Si[2],boundary_low_600Si[3]])
SE_geHot_600_boundary_low = SE_geHot_600_boundary_low/math.sqrt(5)

SE_geHot_800_boundary_low = np.std([boundary_low_800Si[0],boundary_low_800Si[1],boundary_low_800Si[2],boundary_low_800Si[3]])
SE_geHot_800_boundary_low = SE_geHot_800_boundary_low/math.sqrt(5)

SE_geHot_1000_boundary_low = np.std([boundary_low_1000Si[0],boundary_low_1000Si[1],boundary_low_1000Si[2],boundary_low_1000Si[3]])
SE_geHot_1000_boundary_low = SE_geHot_1000_boundary_low/math.sqrt(5)

#SE for bottom boundary temperatures by color
SE_siHot_r50_boundary_low = np.std([boundary_low_200Ge[0],boundary_low_400Ge[0],boundary_low_600Ge[0],boundary_low_800Ge[0],boundary_low_1000Ge[0]])
SE_siHot_r50_boundary_low = SE_siHot_r50_boundary_low/math.sqrt(5)

SE_siHot_r125_boundary_low = np.std([boundary_low_200Ge[1],boundary_low_400Ge[1],boundary_low_600Ge[1],boundary_low_800Ge[1],boundary_low_1000Ge[1]])
SE_siHot_r125_boundary_low = SE_siHot_r125_boundary_low/math.sqrt(5)

SE_siHot_r250_boundary_low = np.std([boundary_low_200Ge[2],boundary_low_400Ge[2],boundary_low_600Ge[2],boundary_low_800Ge[2],boundary_low_1000Ge[2]])
SE_siHot_r250_boundary_low = SE_siHot_r250_boundary_low/math.sqrt(5)

SE_siHot_r271_boundary_low = np.std([boundary_low_200Ge[3],boundary_low_400Ge[3],boundary_low_600Ge[3],boundary_low_800Ge[3],boundary_low_1000Ge[3]])
SE_siHot_r271_boundary_low = SE_siHot_r271_boundary_low/math.sqrt(5)


#Average boundary_low error by size
SE_siHot_200_boundary_low = np.std([boundary_low_200Ge[0],boundary_low_200Ge[1],boundary_low_200Ge[2],boundary_low_200Ge[3]])
SE_siHot_200_boundary_low = SE_siHot_200_boundary_low/math.sqrt(5)

SE_siHot_400_boundary_low = np.std([boundary_low_400Ge[0],boundary_low_400Ge[1],boundary_low_400Ge[2],boundary_low_400Ge[3]])
SE_siHot_400_boundary_low = SE_siHot_400_boundary_low/math.sqrt(5)

SE_siHot_600_boundary_low = np.std([boundary_low_600Ge[0],boundary_low_600Ge[1],boundary_low_600Ge[2],boundary_low_600Ge[3]])
SE_siHot_600_boundary_low = SE_siHot_600_boundary_low/math.sqrt(5)

SE_siHot_800_boundary_low = np.std([boundary_low_800Ge[0],boundary_low_800Ge[1],boundary_low_800Ge[2],boundary_low_800Ge[3]])
SE_siHot_800_boundary_low = SE_siHot_800_boundary_low/math.sqrt(5)

SE_siHot_1000_boundary_low = np.std([boundary_low_1000Ge[0],boundary_low_1000Ge[1],boundary_low_1000Ge[2],boundary_low_1000Ge[3]])
SE_siHot_1000_boundary_low = SE_siHot_1000_boundary_low/math.sqrt(5)

In [ ]:
#SE for middle temperatures by color
SE_geHot_r50_mid_temp = np.std([mid_temp_200Si[0],mid_temp_400Si[0],mid_temp_600Si[0],mid_temp_800Si[0],mid_temp_1000Si[0]])
SE_geHot_r50_mid_temp = SE_geHot_r50_mid_temp/math.sqrt(5)

SE_geHot_r125_mid_temp = np.std([mid_temp_200Si[1],mid_temp_400Si[1],mid_temp_600Si[1],mid_temp_800Si[1],mid_temp_1000Si[1]])
SE_geHot_r125_mid_temp = SE_geHot_r125_mid_temp/math.sqrt(5)

SE_geHot_r250_mid_temp = np.std([mid_temp_200Si[2],mid_temp_400Si[2],mid_temp_600Si[2],mid_temp_800Si[2],mid_temp_1000Si[2]])
SE_geHot_r250_mid_temp = SE_geHot_r250_mid_temp/math.sqrt(5)

SE_geHot_r271_mid_temp = np.std([mid_temp_200Si[3],mid_temp_400Si[3],mid_temp_600Si[3],mid_temp_800Si[3],mid_temp_1000Si[3]])
SE_geHot_r271_mid_temp = SE_geHot_r271_mid_temp/math.sqrt(5)


#Average mid_temp error by size
SE_geHot_200_mid_temp = np.std([mid_temp_200Si[0],mid_temp_200Si[1],mid_temp_200Si[2],mid_temp_200Si[3]])
SE_geHot_200_mid_temp = SE_geHot_200_mid_temp/math.sqrt(5)

SE_geHot_400_mid_temp = np.std([mid_temp_400Si[0],mid_temp_400Si[1],mid_temp_400Si[2],mid_temp_400Si[3]])
SE_geHot_400_mid_temp = SE_geHot_400_mid_temp/math.sqrt(5)

SE_geHot_600_mid_temp = np.std([mid_temp_600Si[0],mid_temp_600Si[1],mid_temp_600Si[2],mid_temp_600Si[3]])
SE_geHot_600_mid_temp = SE_geHot_600_mid_temp/math.sqrt(5)

SE_geHot_800_mid_temp = np.std([mid_temp_800Si[0],mid_temp_800Si[1],mid_temp_800Si[2],mid_temp_800Si[3]])
SE_geHot_800_mid_temp = SE_geHot_800_mid_temp/math.sqrt(5)

SE_geHot_1000_mid_temp = np.std([mid_temp_1000Si[0],mid_temp_1000Si[1],mid_temp_1000Si[2],mid_temp_1000Si[3]])
SE_geHot_1000_mid_temp = SE_geHot_1000_mid_temp/math.sqrt(5)

#SE for middle temperatures by color
SE_siHot_r50_mid_temp = np.std([mid_temp_200Ge[0],mid_temp_400Ge[0],mid_temp_600Ge[0],mid_temp_800Ge[0],mid_temp_1000Ge[0]])
SE_siHot_r50_mid_temp = SE_siHot_r50_mid_temp/math.sqrt(5)

SE_siHot_r125_mid_temp = np.std([mid_temp_200Ge[1],mid_temp_400Ge[1],mid_temp_600Ge[1],mid_temp_800Ge[1],mid_temp_1000Ge[1]])
SE_siHot_r125_mid_temp = SE_siHot_r125_mid_temp/math.sqrt(5)

SE_siHot_r250_mid_temp = np.std([mid_temp_200Ge[2],mid_temp_400Ge[2],mid_temp_600Ge[2],mid_temp_800Ge[2],mid_temp_1000Ge[2]])
SE_siHot_r250_mid_temp = SE_siHot_r250_mid_temp/math.sqrt(5)

SE_siHot_r271_mid_temp = np.std([mid_temp_200Ge[3],mid_temp_400Ge[3],mid_temp_600Ge[3],mid_temp_800Ge[3],mid_temp_1000Ge[3]])
SE_siHot_r271_mid_temp = SE_siHot_r271_mid_temp/math.sqrt(5)


#Average mid_temp error by size
SE_siHot_200_mid_temp = np.std([mid_temp_200Ge[0],mid_temp_200Ge[1],mid_temp_200Ge[2],mid_temp_200Ge[3]])
SE_siHot_200_mid_temp = SE_siHot_200_mid_temp/math.sqrt(5)

SE_siHot_400_mid_temp = np.std([mid_temp_400Ge[0],mid_temp_400Ge[1],mid_temp_400Ge[2],mid_temp_400Ge[3]])
SE_siHot_400_mid_temp = SE_siHot_400_mid_temp/math.sqrt(5)

SE_siHot_600_mid_temp = np.std([mid_temp_600Ge[0],mid_temp_600Ge[1],mid_temp_600Ge[2],mid_temp_600Ge[3]])
SE_siHot_600_mid_temp = SE_siHot_600_mid_temp/math.sqrt(5)

SE_siHot_800_mid_temp = np.std([mid_temp_800Ge[0],mid_temp_800Ge[1],mid_temp_800Ge[2],mid_temp_800Ge[3]])
SE_siHot_800_mid_temp = SE_siHot_800_mid_temp/math.sqrt(5)

SE_siHot_1000_mid_temp = np.std([mid_temp_1000Ge[0],mid_temp_1000Ge[1],mid_temp_1000Ge[2],mid_temp_1000Ge[3]])
SE_siHot_1000_mid_temp = SE_siHot_1000_mid_temp/math.sqrt(5)

In [ ]:
#SIGESI
Int_R_SI200 = [np.mean(R_int_s200r50),np.mean(R_int_s200r125),np.mean(R_int_s200r250),np.mean(R_int_s200r271)]
Int_R_SI400 = [np.mean(R_int_s400r50),np.mean(R_int_s400r125),np.mean(R_int_s400r250),np.mean(R_int_s400r271)]
Int_R_SI600 = [np.mean(R_int_s600r50),np.mean(R_int_s600r125),np.mean(R_int_s600r250),np.mean(R_int_s600r271)]
Int_R_SI800 = [np.mean(R_int_s800r50),np.mean(R_int_s800r125),np.mean(R_int_s800r250),np.mean(R_int_s800r271)]
Int_R_SI1000 = [np.mean(R_int_s1000r50),np.mean(R_int_s1000r125),np.mean(R_int_s1000r250),np.mean(R_int_s1000r271)]

INT_R_SI_ALL = [np.mean(Int_R_SI200),np.mean(Int_R_SI400),np.mean(Int_R_SI600),np.mean(Int_R_SI800),np.mean(Int_R_SI1000)]
MID_TEMP_SI_ALL = [np.mean(mid_temp_200Si),np.mean(mid_temp_400Si),np.mean(mid_temp_600Si),np.mean(mid_temp_800Si),np.mean(mid_temp_1000Si)]
TEMP_SI_ALL = [np.mean(temp_D_200Si),np.mean(temp_D_400Si),np.mean(temp_D_600Si),np.mean(temp_D_800Si),np.mean(temp_D_1000Si)]
BOUNDARY_LOW_SI_ALL = [np.mean(boundary_low_200Si),np.mean(boundary_low_400Si),np.mean(boundary_low_600Si),np.mean(boundary_low_800Si),np.mean(boundary_low_1000Si)]
BOUNDARY_HIGH_SI_ALL = [np.mean(boundary_high_200Si),np.mean(boundary_high_400Si),np.mean(boundary_high_600Si),np.mean(boundary_high_800Si),np.mean(boundary_high_1000Si)]


#GESIGE
Int_R_GE200 = [np.mean(R_int_g200r50),np.mean(R_int_g200r125),np.mean(R_int_g200r250),np.mean(R_int_g200r271)]
Int_R_GE400 = [np.mean(R_int_g400r50),np.mean(R_int_g400r125),np.mean(R_int_g400r250),np.mean(R_int_g400r271)]
Int_R_GE600 = [np.mean(R_int_g600r50),np.mean(R_int_g600r125),np.mean(R_int_g600r250),np.mean(R_int_g600r271)]
Int_R_GE800 = [np.mean(R_int_g800r50),np.mean(R_int_g800r125),np.mean(R_int_g800r250),np.mean(R_int_g800r271)]
Int_R_GE1000 = [np.mean(R_int_g1000r50),np.mean(R_int_g1000r125),np.mean(R_int_g1000r250),np.mean(R_int_g1000r271)]

INT_R_GE_ALL = [np.mean(Int_R_GE200),np.mean(Int_R_GE400),np.mean(Int_R_GE600),np.mean(Int_R_GE800),np.mean(Int_R_GE1000)]#,Int_R_GE1500,Int_R_GE2k]
MID_TEMP_GE_ALL = [np.mean(mid_temp_200Ge),np.mean(mid_temp_400Ge),np.mean(mid_temp_600Ge),np.mean(mid_temp_800Ge),np.mean(mid_temp_1000Ge)]#,mid_temp_1500Ge,mid_temp_2kGe]
TEMP_GE_ALL = [np.mean(temp_D_200Ge),np.mean(temp_D_400Ge),np.mean(temp_D_600Ge),np.mean(temp_D_800Ge),np.mean(temp_D_1000Ge)]
BOUNDARY_LOW_GE_ALL = [np.mean(boundary_low_200Ge),np.mean(boundary_low_400Ge),np.mean(boundary_low_600Ge),np.mean(boundary_low_800Ge),np.mean(boundary_low_1000Ge)]
BOUNDARY_HIGH_GE_ALL = [np.mean(boundary_high_200Ge),np.mean(boundary_high_400Ge),np.mean(boundary_high_600Ge),np.mean(boundary_high_800Ge),np.mean(boundary_high_1000Ge)]



In [ ]:
rad_size1 = np.ones(4)*110.0
rad_size2 = np.ones(4)*219.9
rad_size3 = np.ones(4)*329.9
rad_size4 = np.ones(4)*439.9
rad_size5 = np.ones(4)*550.0
rad_size = [np.mean(rad_size1),np.mean(rad_size2),np.mean(rad_size3),np.mean(rad_size4),np.mean(rad_size5)]

rad_sizeG1 = np.ones(4)*110.0
rad_sizeG2 = np.ones(4)*219.9
rad_sizeG3 = np.ones(4)*329.9
rad_sizeG4 = np.ones(4)*439.9
rad_sizeG5 = np.ones(4)*550.0
rad_sizeG = [np.mean(rad_sizeG1),np.mean(rad_sizeG2),np.mean(rad_sizeG3),np.mean(rad_sizeG4),np.mean(rad_sizeG5)]


In [ ]:
#size
all_boxes_si_size = []
for t in rad_size:
    all_boxes_si_size.append(t)
print(np.shape(all_boxes_si_size))
print(all_boxes_si_size)

all_boxes_ge_size = []
for t in rad_sizeG:
    all_boxes_ge_size.append(t)

#IntR
all_boxes_si_IntR = []
for t in INT_R_SI_ALL:
    all_boxes_si_IntR.append(t)
all_boxes_si_IntR = [i/1e8 for i in all_boxes_si_IntR]
print(np.shape(all_boxes_si_IntR))

all_boxes_ge_IntR = []
for t in INT_R_GE_ALL:
    all_boxes_ge_IntR.append(t)
all_boxes_ge_IntR = [i/1e8 for i in all_boxes_ge_IntR]

#midT
all_boxes_si_midT = []
for t in MID_TEMP_SI_ALL:
    all_boxes_si_midT.append(t)

all_boxes_ge_midT = []
for t in MID_TEMP_GE_ALL:
    all_boxes_ge_midT.append(t)

#TempD
all_boxes_si_TempD = []
for t in TEMP_SI_ALL:
    all_boxes_si_TempD.append(t)

all_boxes_ge_TempD = []
for t in TEMP_GE_ALL:
    all_boxes_ge_TempD.append(t)

#upperBoundary
all_boxes_si_upperBoundary = []
for t in BOUNDARY_HIGH_SI_ALL:
    all_boxes_si_upperBoundary.append(t)

all_boxes_ge_upperBoundary = []
for t in BOUNDARY_HIGH_GE_ALL:
    all_boxes_ge_upperBoundary.append(t)

#lowerBoundary
all_boxes_si_lowerBoundary = []
for t in BOUNDARY_LOW_SI_ALL:
    all_boxes_si_lowerBoundary.append(t)

all_boxes_ge_lowerBoundary = []
for t in BOUNDARY_LOW_GE_ALL:
    all_boxes_ge_lowerBoundary.append(t)


In [ ]:
#2D plots

In [ ]:
#size vs Int_R

In [ ]:
#from mpl_toolkits.mplot3d import axes3d, Axes3D
#from scipy.interpolate import griddata


fig, ax = plt.subplots(figsize=(12,8))

xline_si = rad_size
zline_si = INT_R_SI_ALL
yline_si = MID_TEMP_SI_ALL
xline_si = np.array(xline_si); yline_si = np.array(yline_si); zline_si = np.array(zline_si);  

xline = rad_sizeG
zline = INT_R_GE_ALL
yline = MID_TEMP_GE_ALL
xline = np.array(xline); yline = np.array(yline); zline = np.array(zline);  

av_size_Si200 = np.mean(xline_si[:][0])
av_Int_Si200 = np.mean(zline_si[:][0])
av_midT_Si200 = np.mean(yline_si[:][0])
av_size_Si400 = np.mean(xline_si[:][1])
av_Int_Si400 = np.mean(zline_si[:][1])
av_midT_Si400 = np.mean(yline_si[:][1])
av_size_Si600 = np.mean(xline_si[:][2])
av_Int_Si600 = np.mean(zline_si[:][2])
av_midT_Si600 = np.mean(yline_si[:][2])
av_size_Si800 = np.mean(xline_si[:][3])
av_Int_Si800 = np.mean(zline_si[:][3])
av_midT_Si800 = np.mean(yline_si[:][3])
av_size_Si1000 = np.mean(xline_si[:][4])
av_Int_Si1000 = np.mean(zline_si[:][4])
av_midT_Si1000 = np.mean(yline_si[:][4])

av_size_Ge200 = np.mean(xline[:][0])
av_Int_Ge200 = np.mean(zline[:][0])
av_midT_Ge200 = np.mean(yline[:][0])
av_size_Ge400 = np.mean(xline[:][1])
av_Int_Ge400 = np.mean(zline[:][1])
av_midT_Ge400 = np.mean(yline[:][1])
av_size_Ge600 = np.mean(xline[:][2])
av_Int_Ge600 = np.mean(zline[:][2])
av_midT_Ge600 = np.mean(yline[:][2])
av_size_Ge800 = np.mean(xline[:][3])
av_Int_Ge800 = np.mean(zline[:][3])
av_midT_Ge800 = np.mean(yline[:][3])
av_size_Ge1000 = np.mean(xline[:][4])
av_Int_Ge1000 = np.mean(zline[:][4])
av_midT_Ge1000 = np.mean(yline[:][4])


#Ge(hot)
plt.scatter(av_size_Si200, av_Int_Si200/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100,label='Ge(hot)-200')
#print(av_Int_Si200/1e8)
#print(SE_geHot_200_rint/1e8)
plt.scatter(av_size_Si400, av_Int_Si400/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100,label='Ge(hot)-400')
plt.scatter(av_size_Si600, av_Int_Si600/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100,label='Ge(hot)-600')
plt.scatter(av_size_Si800, av_Int_Si800/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100,label='Ge(hot)-800')
plt.scatter(av_size_Si1000, av_Int_Si1000/1e8,facecolors=color_blind_safe[8], edgecolors=color_blind_safe[8],marker="o",s=100,label='Ge(hot)-1000')


#Si(hot)
plt.scatter(av_size_Ge200, av_Int_Ge200/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100,label='Si(hot)-200')
plt.scatter(av_size_Ge400, av_Int_Ge400/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100,label='Si(hot)-400')
plt.scatter(av_size_Ge600, av_Int_Ge600/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100,label='Si(hot)-600')
plt.scatter(av_size_Ge800, av_Int_Ge800/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100,label='Si(hot)-800')
plt.scatter(av_size_Ge1000, av_Int_Ge1000/1e8,facecolors=color_blind_safe[8], edgecolors=color_blind_safe[8],marker="s",s=100,label='Si(hot)-1000')

plt.errorbar(av_size_Si200, av_Int_Si200/1e8, SE_geHot_200_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[1],color=color_blind_safe[1])
plt.errorbar(av_size_Si400, av_Int_Si400/1e8, SE_geHot_400_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[2],color=color_blind_safe[2])
plt.errorbar(av_size_Si600, av_Int_Si600/1e8, SE_geHot_600_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[3],color=color_blind_safe[3])
plt.errorbar(av_size_Si800, av_Int_Si800/1e8, SE_geHot_800_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[4],color=color_blind_safe[4])
plt.errorbar(av_size_Si1000, av_Int_Si1000/1e8, SE_geHot_1000_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[8],color=color_blind_safe[8])

plt.errorbar(av_size_Ge200, av_Int_Ge200/1e8, SE_siHot_200_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[1],color=color_blind_safe[1])
plt.errorbar(av_size_Ge400, av_Int_Ge400/1e8, SE_siHot_400_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[2],color=color_blind_safe[2])
plt.errorbar(av_size_Ge600, av_Int_Ge600/1e8, SE_siHot_600_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[3],color=color_blind_safe[3])
plt.errorbar(av_size_Ge800, av_Int_Ge800/1e8, SE_siHot_800_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[4],color=color_blind_safe[4])
plt.errorbar(av_size_Ge1000, av_Int_Ge1000/1e8, SE_siHot_1000_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[8],color=color_blind_safe[8])

print(SE_geHot_200_rint)

#Linear fit
from scipy import stats
print(all_boxes_ge_size)

mnG = min(all_boxes_ge_size)
mxG = max(all_boxes_ge_size)

mnS = min(all_boxes_si_size)
mxS = max(all_boxes_si_size)


#SiGeSi
#using numpy
p30_Si = np.poly1d(np.polyfit(all_boxes_si_size, all_boxes_si_IntR, 3))
xp_p30_Si = np.linspace(-0.001, mxS+0.001,5)
slope_intercept_p30_Si = np.polyfit(all_boxes_si_size, all_boxes_si_IntR, 1)
#print('Slope, Intercept of p30_Si numpy:',slope_intercept)


polyfit_fit_Si = np.polyfit(all_boxes_si_size, all_boxes_si_IntR, 1) 
polyfit_fit_Si = np.poly1d(polyfit_fit_Si)
xp_Si = np.linspace(mnS-0.001, mxS+0.001, 20)  
slope_intercept_Si = np.polyfit(all_boxes_si_size, all_boxes_si_IntR, 1)
#print('Slope, Intercept of Si polyfit numpy:',slope_intercept_Si)


#GeSiGe
#using numpy
p30_Ge = np.poly1d(np.polyfit(all_boxes_ge_size, all_boxes_ge_IntR, 3))
xp_p30_Ge = np.linspace(-0.001, mxG+0.001,5)
slope_intercept_p30_Ge = np.polyfit(all_boxes_ge_size, all_boxes_ge_IntR, 1)
#print('Slope, Intercept of p30_Ge numpy:',slope_intercept)


polyfit_fit_Ge = np.polyfit(all_boxes_ge_size, all_boxes_ge_IntR, 1) 
polyfit_fit_Ge = np.poly1d(polyfit_fit_Ge)
#print('Polyfit Ge:',polyfit_fit_Ge)
xp_Ge = np.linspace(mnG-0.001, mxG+0.001,20)  #xp = np.linspace(-0.001, mx+0.001, 5)
slope_intercept_Ge = np.polyfit(all_boxes_ge_size, all_boxes_ge_IntR, 1)
#print('Slope, Intercept of Ge polyfit numpy:',slope_intercept_Ge)


bbox = dict(boxstyle ="round", fc ="0.9") 


plt.plot(xp_Si, polyfit_fit_Si(xp_Si), '--', color ='blue', label = 'Ge(hot)')
plt.plot(xp_Ge, polyfit_fit_Ge(xp_Ge), '--', color ='black', label = 'Si(hot)')
plt.text(110, 3.8,"y=%.6fx+%.6f"%(polyfit_fit_Si[1],polyfit_fit_Si[0]))
plt.text(110, 2.8,"y=%.6fx+%.6f"%(polyfit_fit_Ge[1],polyfit_fit_Ge[0]))


#using scipy
res_Si = stats.linregress(all_boxes_si_size, all_boxes_si_IntR)
print(f"R-squared Si scipy: {res_Si.rvalue**2:.6f}")
print(f"Intercept Si scipy: {res_Si.intercept:.6f}")
print(f"Slope Si scipy: {res_Si.slope:.6f}")

res_Ge = stats.linregress(all_boxes_ge_size, all_boxes_ge_IntR)
print(f"R-squared Ge scipy: {res_Ge.rvalue**2:.6f}")
print(f"Intercept Ge scipy: {res_Ge.intercept:.6f}")
print(f"Slope Ge scipy: {res_Ge.slope:.6f}")

ax.annotate('$R^2$ Ge(hot): = %.6f' % (res_Si.rvalue**2),
            xy=(110, 4.1), xytext=(110, 4.1), bbox = bbox)

ax.annotate('$R^2$ Si(hot): = %.6f' % (res_Ge.rvalue**2),
            xy=(110.0, 3.1), xytext=(110, 3.1), bbox = bbox)

#Linear fit

plt.legend(bbox_to_anchor=(1.1, 1.0), loc='upper left',labelspacing=0.6, fontsize=20) 
plt.xlabel('\n\n Length (nm)',fontsize=27)
plt.ylabel('\n\n $R_{int}$ ($10^8$ K/W)',fontsize=27)

plt.xticks([110.0, 219.9, 329.9, 439.9, 550.0],
           ['110.0','219.9','329.9','439.9','550.0'],rotation=0) 


plt.show()  


In [ ]:
#Size vs Mid.temp

In [ ]:
#from mpl_toolkits.mplot3d import axes3d, Axes3D
#from scipy.interpolate import griddata


fig, ax = plt.subplots(figsize=(12,8))


xline_si = rad_size
yline_si = MID_TEMP_SI_ALL
zline_si = INT_R_SI_ALL
xline_si = np.array(xline_si); yline_si = np.array(yline_si); zline_si = np.array(zline_si);  

xline = rad_sizeG
yline = MID_TEMP_GE_ALL
zline = INT_R_GE_ALL
xline = np.array(xline); yline = np.array(yline); zline = np.array(zline);  

av_size_Si200 = np.mean(xline_si[:][0])
av_Int_Si200 = np.mean(zline_si[:][0])
av_midT_Si200 = np.mean(yline_si[:][0])
av_size_Si400 = np.mean(xline_si[:][1])
av_Int_Si400 = np.mean(zline_si[:][1])
av_midT_Si400 = np.mean(yline_si[:][1])
av_size_Si600 = np.mean(xline_si[:][2])
av_Int_Si600 = np.mean(zline_si[:][2])
av_midT_Si600 = np.mean(yline_si[:][2])
av_size_Si800 = np.mean(xline_si[:][3])
av_Int_Si800 = np.mean(zline_si[:][3])
av_midT_Si800 = np.mean(yline_si[:][3])
av_size_Si1000 = np.mean(xline_si[:][4])
av_Int_Si1000 = np.mean(zline_si[:][4])
av_midT_Si1000 = np.mean(yline_si[:][4])

av_size_Ge200 = np.mean(xline[:][0])
av_Int_Ge200 = np.mean(zline[:][0])
av_midT_Ge200 = np.mean(yline[:][0])
av_size_Ge400 = np.mean(xline[:][1])
av_Int_Ge400 = np.mean(zline[:][1])
av_midT_Ge400 = np.mean(yline[:][1])
av_size_Ge600 = np.mean(xline[:][2])
av_Int_Ge600 = np.mean(zline[:][2])
av_midT_Ge600 = np.mean(yline[:][2])
av_size_Ge800 = np.mean(xline[:][3])
av_Int_Ge800 = np.mean(zline[:][3])
av_midT_Ge800 = np.mean(yline[:][3])
av_size_Ge1000 = np.mean(xline[:][4])
av_Int_Ge1000 = np.mean(zline[:][4])
av_midT_Ge1000 = np.mean(yline[:][4])


#Ge(hot)
plt.scatter(av_size_Si200, av_midT_Si200,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100,label='Ge(hot)-200')
plt.scatter(av_size_Si400, av_midT_Si400,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100,label='Ge(hot)-400')
plt.scatter(av_size_Si600, av_midT_Si600,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100,label='Ge(hot)-600')
plt.scatter(av_size_Si800, av_midT_Si800,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100,label='Ge(hot)-800')
plt.scatter(av_size_Si1000, av_midT_Si1000,facecolors=color_blind_safe[8], edgecolors=color_blind_safe[8],marker="o",s=100,label='Ge(hot)-1000')


#Si(hot)
plt.scatter(av_size_Ge200, av_midT_Ge200,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100,label='Si(hot)-200')
plt.scatter(av_size_Ge400, av_midT_Ge400,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100,label='Si(hot)-400')
plt.scatter(av_size_Ge600, av_midT_Ge600,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100,label='Si(hot)-600')
plt.scatter(av_size_Ge800, av_midT_Ge800,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100,label='Si(hot)-800')
plt.scatter(av_size_Ge1000, av_midT_Ge1000,facecolors=color_blind_safe[8], edgecolors=color_blind_safe[8],marker="s",s=100,label='Si(hot)-1000')

#Linear fit
from scipy import stats

mnG = min(all_boxes_ge_size)
mxG = max(all_boxes_ge_size)

mnS = min(all_boxes_si_size)
mxS = max(all_boxes_si_size)

#SiGeSi
#using numpy
p30_Si = np.poly1d(np.polyfit(all_boxes_si_size, all_boxes_si_midT, 3))
xp_p30_Si = np.linspace(-0.001, mxS+0.001,5)
slope_intercept_p30_Si = np.polyfit(all_boxes_si_size, all_boxes_si_midT, 1)
#print('Slope, Intercept of p30_Si numpy:',slope_intercept)


polyfit_fit_Si = np.polyfit(all_boxes_si_size, all_boxes_si_midT, 1) 
polyfit_fit_Si = np.poly1d(polyfit_fit_Si)
xp_Si = np.linspace(mnS-0.001, mxS+0.001, 20)  
slope_intercept_Si = np.polyfit(all_boxes_si_size, all_boxes_si_midT, 1)
#print('Slope, Intercept of Si polyfit numpy:',slope_intercept_Si)


#GeSiGe
#using numpy
p30_Ge = np.poly1d(np.polyfit(all_boxes_ge_size, all_boxes_ge_midT, 3))
xp_p30_Ge = np.linspace(-0.001, mxG+0.001,5)
slope_intercept_p30_Ge = np.polyfit(all_boxes_ge_size, all_boxes_ge_midT, 1)
#print('Slope, Intercept of p30_Ge numpy:',slope_intercept)


polyfit_fit_Ge = np.polyfit(all_boxes_ge_size, all_boxes_ge_midT, 1) 
polyfit_fit_Ge = np.poly1d(polyfit_fit_Ge)
#print('Polyfit Ge:',polyfit_fit_Ge)
xp_Ge = np.linspace(mnG-0.001, mxG+0.001,20)  #xp = np.linspace(-0.001, mx+0.001, 5)
slope_intercept_Ge = np.polyfit(all_boxes_ge_size, all_boxes_ge_midT, 1)
#print('Slope, Intercept of Ge polyfit numpy:',slope_intercept_Ge)


bbox = dict(boxstyle ="round", fc ="0.9") 


plt.plot(xp_Si, polyfit_fit_Si(xp_Si), '--', color ='blue', label = 'Ge(hot)')
plt.plot(xp_Ge, polyfit_fit_Ge(xp_Ge), '--', color ='black', label = 'Si(hot)')
plt.text(110, 337.5,"y=%.6fx+%.6f"%(polyfit_fit_Si[1],polyfit_fit_Si[0]))
plt.text(329, 342.5,"y=%.6fx+%.6f"%(polyfit_fit_Ge[1],polyfit_fit_Ge[0]))


#using scipy
res_Si = stats.linregress(all_boxes_si_size, all_boxes_si_midT)
print(f"R-squared Si scipy: {res_Si.rvalue**2:.6f}")
print(f"Intercept Si scipy: {res_Si.intercept:.6f}")
print(f"Slope Si scipy: {res_Si.slope:.6f}")

res_Ge = stats.linregress(all_boxes_ge_size, all_boxes_ge_midT)
print(f"R-squared Ge scipy: {res_Ge.rvalue**2:.6f}")
print(f"Intercept Ge scipy: {res_Ge.intercept:.6f}")
print(f"Slope Ge scipy: {res_Ge.slope:.6f}")

ax.annotate('$R^2$ Ge(hot): = %.6f' % (res_Si.rvalue**2),
            xy=(110, 335), xytext=(110, 335), bbox = bbox)

ax.annotate('$R^2$ Si(hot): = %.6f' % (res_Ge.rvalue**2),
            xy=(329, 340), xytext=(329, 340), bbox = bbox)

#Linear fit

plt.legend(bbox_to_anchor=(1.1, 1.0), loc='upper left',labelspacing=0.6, fontsize=20) 
plt.xlabel('\n\n Length (nm)',fontsize=27)
plt.ylabel('\n\n Middle Temperature (K)',fontsize=27)

plt.xticks([110.0, 219.9, 329.9, 439.9, 550.0],
           ['110.0','219.9','329.9','439.9','550.0'],rotation=0) 


plt.show()  

In [ ]:
#Mid.temp vs Int_R

In [ ]:
#from mpl_toolkits.mplot3d import axes3d, Axes3D
#from scipy.interpolate import griddata


fig, ax = plt.subplots(figsize=(12,8))

xline_si = rad_size
yline_si = MID_TEMP_SI_ALL
zline_si = INT_R_SI_ALL
xline_si = np.array(xline_si); yline_si = np.array(yline_si); zline_si = np.array(zline_si);  

xline = rad_sizeG
yline = MID_TEMP_GE_ALL
zline = INT_R_GE_ALL
xline = np.array(xline); yline = np.array(yline); zline = np.array(zline);  

av_size_Si200 = np.mean(xline_si[:][0])
av_Int_Si200 = np.mean(zline_si[:][0])
av_midT_Si200 = np.mean(yline_si[:][0])
av_size_Si400 = np.mean(xline_si[:][1])
av_Int_Si400 = np.mean(zline_si[:][1])
av_midT_Si400 = np.mean(yline_si[:][1])
av_size_Si600 = np.mean(xline_si[:][2])
av_Int_Si600 = np.mean(zline_si[:][2])
av_midT_Si600 = np.mean(yline_si[:][2])
av_size_Si800 = np.mean(xline_si[:][3])
av_Int_Si800 = np.mean(zline_si[:][3])
av_midT_Si800 = np.mean(yline_si[:][3])
av_size_Si1000 = np.mean(xline_si[:][4])
av_Int_Si1000 = np.mean(zline_si[:][4])
av_midT_Si1000 = np.mean(yline_si[:][4])

av_size_Ge200 = np.mean(xline[:][0])
av_Int_Ge200 = np.mean(zline[:][0])
av_midT_Ge200 = np.mean(yline[:][0])
av_size_Ge400 = np.mean(xline[:][1])
av_Int_Ge400 = np.mean(zline[:][1])
av_midT_Ge400 = np.mean(yline[:][1])
av_size_Ge600 = np.mean(xline[:][2])
av_Int_Ge600 = np.mean(zline[:][2])
av_midT_Ge600 = np.mean(yline[:][2])
av_size_Ge800 = np.mean(xline[:][3])
av_Int_Ge800 = np.mean(zline[:][3])
av_midT_Ge800 = np.mean(yline[:][3])
av_size_Ge1000 = np.mean(xline[:][4])
av_Int_Ge1000 = np.mean(zline[:][4])
av_midT_Ge1000 = np.mean(yline[:][4])


#Ge(hot)
plt.scatter(av_midT_Si200, av_Int_Si200/1e8, facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100,label='Ge(hot)-200')
plt.scatter(av_midT_Si400, av_Int_Si400/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100,label='Ge(hot)-400')
plt.scatter(av_midT_Si600, av_Int_Si600/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100,label='Ge(hot)-600')
plt.scatter(av_midT_Si800, av_Int_Si800/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100,label='Ge(hot)-800')
plt.scatter(av_midT_Si1000, av_Int_Si1000/1e8,facecolors=color_blind_safe[8], edgecolors=color_blind_safe[8],marker="o",s=100,label='Ge(hot)-1000')


#Si(hot)
plt.scatter(av_midT_Ge200, av_Int_Ge200/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100,label='Si(hot)-200')
plt.scatter(av_midT_Ge400, av_Int_Ge400/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100,label='Si(hot)-400')
plt.scatter(av_midT_Ge600, av_Int_Ge600/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100,label='Si(hot)-600')
plt.scatter(av_midT_Ge800, av_Int_Ge800/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100,label='Si(hot)-800')
plt.scatter(av_midT_Ge1000, av_Int_Ge1000/1e8,facecolors=color_blind_safe[8], edgecolors=color_blind_safe[8],marker="s",s=100,label='Si(hot)-1000')

plt.errorbar(av_midT_Si200, av_Int_Si200/1e8, xerr=SE_geHot_200_mid_temp, yerr= SE_geHot_200_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[1],color=color_blind_safe[1])
plt.errorbar(av_midT_Si400, av_Int_Si400/1e8, xerr=SE_geHot_400_mid_temp, yerr= SE_geHot_400_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[2],color=color_blind_safe[2])
plt.errorbar(av_midT_Si600, av_Int_Si600/1e8, xerr=SE_geHot_600_mid_temp, yerr= SE_geHot_600_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[3],color=color_blind_safe[3])
plt.errorbar(av_midT_Si800, av_Int_Si800/1e8, xerr=SE_geHot_800_mid_temp, yerr= SE_geHot_800_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[4],color=color_blind_safe[4])
plt.errorbar(av_midT_Si1000, av_Int_Si1000/1e8, xerr=SE_geHot_1000_mid_temp, yerr= SE_geHot_1000_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[8],color=color_blind_safe[8])

plt.errorbar(av_midT_Ge200, av_Int_Ge200/1e8, xerr=SE_siHot_200_mid_temp, yerr= SE_siHot_200_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[1],color=color_blind_safe[1])
plt.errorbar(av_midT_Ge400, av_Int_Ge400/1e8, xerr=SE_siHot_400_mid_temp, yerr= SE_siHot_400_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[2],color=color_blind_safe[2])
plt.errorbar(av_midT_Ge600, av_Int_Ge600/1e8, xerr=SE_siHot_600_mid_temp, yerr= SE_siHot_600_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[3],color=color_blind_safe[3])
plt.errorbar(av_midT_Ge800, av_Int_Ge800/1e8, xerr=SE_siHot_800_mid_temp, yerr= SE_siHot_800_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[4],color=color_blind_safe[4])
plt.errorbar(av_midT_Ge1000, av_Int_Ge1000/1e8, xerr=SE_siHot_1000_mid_temp, yerr= SE_siHot_1000_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[8],color=color_blind_safe[8])

print(SE_geHot_200_rint)
#Linear fit
from scipy import stats

mnG = min(all_boxes_ge_midT)
mxG = max(all_boxes_ge_midT)

mnS = min(all_boxes_si_midT)
mxS = max(all_boxes_si_midT)

#SiGeSi
#using numpy
p30_Si = np.poly1d(np.polyfit(all_boxes_si_midT, all_boxes_si_IntR, 3))
xp_p30_Si = np.linspace(-0.001, mxS+0.001,5)
slope_intercept_p30_Si = np.polyfit(all_boxes_si_midT, all_boxes_si_IntR, 1)
#print('Slope, Intercept of p30_Si numpy:',slope_intercept)


polyfit_fit_Si = np.polyfit(all_boxes_si_midT, all_boxes_si_IntR, 1) 
polyfit_fit_Si = np.poly1d(polyfit_fit_Si)
xp_Si = np.linspace(mnS-0.001, mxS+0.001, 20)  
slope_intercept_Si = np.polyfit(all_boxes_si_midT, all_boxes_si_IntR, 1)
#print('Slope, Intercept of Si polyfit numpy:',slope_intercept_Si)


#GeSiGe
#using numpy
p30_Ge = np.poly1d(np.polyfit(all_boxes_ge_midT, all_boxes_ge_IntR, 3))
xp_p30_Ge = np.linspace(-0.001, mxG+0.001,5)
slope_intercept_p30_Ge = np.polyfit(all_boxes_ge_midT, all_boxes_ge_IntR, 1)
#print('Slope, Intercept of p30_Ge numpy:',slope_intercept)


polyfit_fit_Ge = np.polyfit(all_boxes_ge_midT, all_boxes_ge_IntR, 1) 
polyfit_fit_Ge = np.poly1d(polyfit_fit_Ge)
#print('Polyfit Ge:',polyfit_fit_Ge)
xp_Ge = np.linspace(mnG-0.001, mxG+0.001,20)  #xp = np.linspace(-0.001, mx+0.001, 5)
slope_intercept_Ge = np.polyfit(all_boxes_ge_midT, all_boxes_ge_IntR, 1)
#print('Slope, Intercept of Ge polyfit numpy:',slope_intercept_Ge)


bbox = dict(boxstyle ="round", fc ="0.9") 


plt.plot(xp_Si, polyfit_fit_Si(xp_Si), '--', color ='blue', label = 'Ge(hot)')
plt.plot(xp_Ge, polyfit_fit_Ge(xp_Ge), '--', color ='black', label = 'Si(hot)')
plt.text(330, 2.5,"y=%.6fx+%.6f"%(polyfit_fit_Si[1],polyfit_fit_Si[0]))
plt.text(337, 3.7,"y=%.6fx+%.6f"%(polyfit_fit_Ge[1],polyfit_fit_Ge[0]))


#using scipy
res_Si = stats.linregress(all_boxes_si_midT, all_boxes_si_IntR)
print(f"R-squared Si scipy: {res_Si.rvalue**2:.6f}")
print(f"Intercept Si scipy: {res_Si.intercept:.6f}")
print(f"Slope Si scipy: {res_Si.slope:.6f}")

res_Ge = stats.linregress(all_boxes_ge_midT, all_boxes_ge_IntR)
print(f"R-squared Ge scipy: {res_Ge.rvalue**2:.6f}")
print(f"Intercept Ge scipy: {res_Ge.intercept:.6f}")
print(f"Slope Ge scipy: {res_Ge.slope:.6f}")

ax.annotate('$R^2$ Ge(hot): = %.6f' % (res_Si.rvalue**2),
            xy=(330, 2.8), xytext=(330, 2.8), bbox = bbox)

ax.annotate('$R^2$ Si(hot): = %.6f' % (res_Ge.rvalue**2),
            xy=(337, 4.0), xytext=(337, 4.0), bbox = bbox)

#Linear fit

plt.legend(bbox_to_anchor=(1.1, 1.0), loc='upper left',labelspacing=0.6, fontsize=20) 
plt.ylabel('\n\n $R_{int}$ ($10^8$ K/W)',fontsize=27)
plt.xlabel('\n\n Middle Temperature (K)',fontsize=27)


plt.show()  

In [ ]:
#Rint vs upper boundary temp

In [ ]:
#from mpl_toolkits.mplot3d import axes3d, Axes3D
#from scipy.interpolate import griddata


fig, ax = plt.subplots(figsize=(12,8))


av_boundhigh_Si200 = np.mean(BOUNDARY_HIGH_SI_ALL[:][0])
av_INT_Si200 = np.mean(INT_R_SI_ALL[:][0])
av_boundhigh_Si400 = np.mean(BOUNDARY_HIGH_SI_ALL[:][1])
av_INT_Si400 = np.mean(INT_R_SI_ALL[:][1])
av_boundhigh_Si600 = np.mean(BOUNDARY_HIGH_SI_ALL[:][2])
av_INT_Si600 = np.mean(INT_R_SI_ALL[:][2])
av_boundhigh_Si800 = np.mean(BOUNDARY_HIGH_SI_ALL[:][3])
av_INT_Si800 = np.mean(INT_R_SI_ALL[:][3])
av_boundhigh_Si1000 = np.mean(BOUNDARY_HIGH_SI_ALL[:][4])
av_INT_Si1000 = np.mean(INT_R_SI_ALL[:][4])

av_boundhigh_Ge200 = np.mean(BOUNDARY_HIGH_GE_ALL[:][0])
av_INT_Ge200 = np.mean(INT_R_GE_ALL[:][0])

print(av_boundhigh_Ge200)
av_boundhigh_Ge400 = np.mean(BOUNDARY_HIGH_GE_ALL[:][1])
av_INT_Ge400 = np.mean(INT_R_GE_ALL[:][1])
av_boundhigh_Ge600 = np.mean(BOUNDARY_HIGH_GE_ALL[:][2])
av_INT_Ge600 = np.mean(INT_R_GE_ALL[:][2])
av_boundhigh_Ge800 = np.mean(BOUNDARY_HIGH_GE_ALL[:][3])
av_INT_Ge800 = np.mean(INT_R_GE_ALL[:][3])
av_boundhigh_Ge1000 = np.mean(BOUNDARY_HIGH_GE_ALL[:][4])
av_INT_Ge1000 = np.mean(INT_R_GE_ALL[:][4])


#Ge(hot)
plt.scatter(av_boundhigh_Si200, av_INT_Si200/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100,label='Ge(hot)-200')
plt.scatter(av_boundhigh_Si400, av_INT_Si400/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100,label='Ge(hot)-400')
plt.scatter(av_boundhigh_Si600, av_INT_Si600/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100,label='Ge(hot)-600')
plt.scatter(av_boundhigh_Si800, av_INT_Si800/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100,label='Ge(hot)-800')
plt.scatter(av_boundhigh_Si1000, av_INT_Si1000/1e8,facecolors=color_blind_safe[8], edgecolors=color_blind_safe[8],marker="o",s=100,label='Ge(hot)-1000')

#Si(hot)
plt.scatter(av_boundhigh_Ge200, av_INT_Ge200/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100,label='Si(hot)-200')
plt.scatter(av_boundhigh_Ge400, av_INT_Ge400/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100,label='Si(hot)-400')
plt.scatter(av_boundhigh_Ge600, av_INT_Ge600/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100,label='Si(hot)-600')
plt.scatter(av_boundhigh_Ge800, av_INT_Ge800/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100,label='Si(hot)-800')
plt.scatter(av_boundhigh_Ge1000, av_INT_Ge1000/1e8,facecolors=color_blind_safe[8], edgecolors=color_blind_safe[8],marker="s",s=100,label='Si(hot)-1000')

plt.errorbar(av_boundhigh_Si200, av_Int_Si200/1e8, xerr=SE_geHot_200_boundary_high, yerr= SE_geHot_200_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[1],color=color_blind_safe[1])
plt.errorbar(av_boundhigh_Si400, av_Int_Si400/1e8, xerr=SE_geHot_400_boundary_high, yerr= SE_geHot_400_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[2],color=color_blind_safe[2])
plt.errorbar(av_boundhigh_Si600, av_Int_Si600/1e8, xerr=SE_geHot_600_boundary_high, yerr= SE_geHot_600_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[3],color=color_blind_safe[3])
plt.errorbar(av_boundhigh_Si800, av_Int_Si800/1e8, xerr=SE_geHot_800_boundary_high, yerr= SE_geHot_800_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[4],color=color_blind_safe[4])
plt.errorbar(av_boundhigh_Si1000, av_Int_Si1000/1e8, xerr=SE_geHot_1000_boundary_high, yerr= SE_geHot_1000_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[8],color=color_blind_safe[8])

plt.errorbar(av_boundhigh_Ge200, av_Int_Ge200/1e8, xerr=SE_siHot_200_boundary_high, yerr= SE_siHot_200_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[1],color=color_blind_safe[1])
plt.errorbar(av_boundhigh_Ge400, av_Int_Ge400/1e8, xerr=SE_siHot_400_boundary_high, yerr= SE_siHot_400_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[2],color=color_blind_safe[2])
plt.errorbar(av_boundhigh_Ge600, av_Int_Ge600/1e8, xerr=SE_siHot_600_boundary_high, yerr= SE_siHot_600_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[3],color=color_blind_safe[3])
plt.errorbar(av_boundhigh_Ge800, av_Int_Ge800/1e8, xerr=SE_siHot_800_boundary_high, yerr= SE_siHot_800_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[4],color=color_blind_safe[4])
plt.errorbar(av_boundhigh_Ge1000, av_Int_Ge1000/1e8, xerr=SE_siHot_1000_boundary_high, yerr= SE_siHot_1000_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[8],color=color_blind_safe[8])

#Linear fit
from scipy import stats

mnG = min(all_boxes_ge_upperBoundary)
mxG = max(all_boxes_ge_upperBoundary)

mnS = min(all_boxes_si_upperBoundary)
mxS = max(all_boxes_si_upperBoundary)

#SiGeSi
#using numpy
polyfit_fit_Si = np.polyfit(all_boxes_si_upperBoundary, all_boxes_si_IntR, 1) 
polyfit_fit_Si = np.poly1d(polyfit_fit_Si)
xp_Si = np.linspace(mnS-0.001, mxS+0.001, 20)  
slope_intercept_Si = np.polyfit(all_boxes_si_upperBoundary, all_boxes_si_IntR, 1)
#print('Slope, Intercept of Si polyfit numpy:',slope_intercept_Si)


#GeSiGe
#using numpy
polyfit_fit_Ge = np.polyfit(all_boxes_ge_upperBoundary, all_boxes_ge_IntR, 1) 
polyfit_fit_Ge = np.poly1d(polyfit_fit_Ge)
xp_Ge = np.linspace(mnG-0.001, mxG+0.001,20)  
slope_intercept_Ge = np.polyfit(all_boxes_ge_upperBoundary, all_boxes_ge_IntR, 1)
#print('Slope, Intercept of Ge polyfit numpy:',slope_intercept_Ge)


bbox = dict(boxstyle ="round", fc ="0.9") 

#print(polyfit_fit_Si)
#print(xp_Si)
#print(polyfit_fit_Si(xp_Si))
print(len(all_boxes_ge_upperBoundary))
print(len(all_boxes_ge_IntR))

print(len(all_boxes_si_upperBoundary))
print(len(all_boxes_si_IntR))

plt.plot(xp_Si, polyfit_fit_Si(xp_Si), '--', color ='blue', label = 'Ge(hot)')
plt.plot(xp_Ge, polyfit_fit_Ge(xp_Ge), '--', color ='black', label = 'Si(hot)')
plt.text(372, 2.4,"y=%.6fx+%.6f"%(polyfit_fit_Si[1],polyfit_fit_Si[0]))
plt.text(387, 0.3,"y=%.6fx+%.6f"%(polyfit_fit_Ge[1],polyfit_fit_Ge[0]))


#using scipy
res_Si = stats.linregress(all_boxes_si_upperBoundary, all_boxes_si_IntR)
#print(f"R-squared Si scipy: {res_Si.rvalue**2:.6f}")
#print(f"Intercept Si scipy: {res_Si.intercept:.6f}")
#print(f"Slope Si scipy: {res_Si.slope:.6f}")

res_Ge = stats.linregress(all_boxes_ge_upperBoundary, all_boxes_ge_IntR)
#print(f"R-squared Ge scipy: {res_Ge.rvalue**2:.6f}")
#print(f"Intercept Ge scipy: {res_Ge.intercept:.6f}")
#print(f"Slope Ge scipy: {res_Ge.slope:.6f}")

ax.annotate('$R^2$ Ge(hot): = %.6f' % (res_Si.rvalue**2),
            xy=(372, 2.7), xytext=(372, 2.7), bbox = bbox)

ax.annotate('$R^2$ Si(hot): = %.6f' % (res_Ge.rvalue**2),
            xy=(387, 0.6), xytext=(387, 0.6), bbox = bbox)

#Linear fit

plt.legend(bbox_to_anchor=(1.1, 1.0), loc='upper left',labelspacing=0.6, fontsize=20) 
plt.ylabel('\n\n $R_{int}$ ($10^8$ K/W)',fontsize=27)
plt.xlabel('\n\n Top Boundary Temperature (K)',fontsize=27)


plt.show()  

In [ ]:
#Rint vs lower boundary temp

In [ ]:
#from mpl_toolkits.mplot3d import axes3d, Axes3D
#from scipy.interpolate import griddata


fig, ax = plt.subplots(figsize=(12,8))

av_boundlow_Si200 = np.mean(BOUNDARY_LOW_SI_ALL[:][0])
av_INT_Si200 = np.mean(INT_R_SI_ALL[:][0])
av_boundlow_Si400 = np.mean(BOUNDARY_LOW_SI_ALL[:][1])
av_INT_Si400 = np.mean(INT_R_SI_ALL[:][1])
av_boundlow_Si600 = np.mean(BOUNDARY_LOW_SI_ALL[:][2])
av_INT_Si600 = np.mean(INT_R_SI_ALL[:][2])
av_boundlow_Si800 = np.mean(BOUNDARY_LOW_SI_ALL[:][3])
av_INT_Si800 = np.mean(INT_R_SI_ALL[:][3])
av_boundlow_Si1000 = np.mean(BOUNDARY_LOW_SI_ALL[:][4])
av_INT_Si1000 = np.mean(INT_R_SI_ALL[:][4])

av_boundlow_Ge200 = np.mean(BOUNDARY_LOW_GE_ALL[:][0])
av_INT_Ge200 = np.mean(INT_R_GE_ALL[:][0])
av_boundlow_Ge400 = np.mean(BOUNDARY_LOW_GE_ALL[:][1])
av_INT_Ge400 = np.mean(INT_R_GE_ALL[:][1])
av_boundlow_Ge600 = np.mean(BOUNDARY_LOW_GE_ALL[:][2])
av_INT_Ge600 = np.mean(INT_R_GE_ALL[:][2])
av_boundlow_Ge800 = np.mean(BOUNDARY_LOW_GE_ALL[:][3])
av_INT_Ge800 = np.mean(INT_R_GE_ALL[:][3])
av_boundlow_Ge1000 = np.mean(BOUNDARY_LOW_GE_ALL[:][4])
av_INT_Ge1000 = np.mean(INT_R_GE_ALL[:][4])


#Ge(hot)
plt.scatter(av_boundlow_Si200, av_INT_Si200/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100,label='Ge(hot)-200')
plt.scatter(av_boundlow_Si400, av_INT_Si400/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100,label='Ge(hot)-400')
plt.scatter(av_boundlow_Si600, av_INT_Si600/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100,label='Ge(hot)-600')
plt.scatter(av_boundlow_Si800, av_INT_Si800/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100,label='Ge(hot)-800')
plt.scatter(av_boundlow_Si1000, av_INT_Si1000/1e8,facecolors=color_blind_safe[8], edgecolors=color_blind_safe[8],marker="o",s=100,label='Ge(hot)-1000')


#Si(hot)
plt.scatter(av_boundlow_Ge200, av_INT_Ge200/1e8,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100,label='Si(hot)-200')
plt.scatter(av_boundlow_Ge400, av_INT_Ge400/1e8,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100,label='Si(hot)-400')
plt.scatter(av_boundlow_Ge600, av_INT_Ge600/1e8,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100,label='Si(hot)-600')
plt.scatter(av_boundlow_Ge800, av_INT_Ge800/1e8,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100,label='Si(hot)-800')
plt.scatter(av_boundlow_Ge1000, av_INT_Ge1000/1e8,facecolors=color_blind_safe[8], edgecolors=color_blind_safe[8],marker="s",s=100,label='Si(hot)-1000')

plt.errorbar(av_boundlow_Si200, av_Int_Si200/1e8, xerr=SE_geHot_200_boundary_low, yerr= SE_geHot_200_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[1],color=color_blind_safe[1])
plt.errorbar(av_boundlow_Si400, av_Int_Si400/1e8, xerr=SE_geHot_400_boundary_low, yerr= SE_geHot_400_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[2],color=color_blind_safe[2])
plt.errorbar(av_boundlow_Si600, av_Int_Si600/1e8, xerr=SE_geHot_600_boundary_low, yerr= SE_geHot_600_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[3],color=color_blind_safe[3])
plt.errorbar(av_boundlow_Si800, av_Int_Si800/1e8, xerr=SE_geHot_800_boundary_low, yerr= SE_geHot_800_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[4],color=color_blind_safe[4])
plt.errorbar(av_boundlow_Si1000, av_Int_Si1000/1e8, xerr=SE_geHot_1000_boundary_low, yerr= SE_geHot_1000_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[8],color=color_blind_safe[8])

plt.errorbar(av_boundlow_Ge200, av_Int_Ge200/1e8, xerr=SE_siHot_200_boundary_low, yerr= SE_siHot_200_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[1],color=color_blind_safe[1])
plt.errorbar(av_boundlow_Ge400, av_Int_Ge400/1e8, xerr=SE_siHot_400_boundary_low, yerr= SE_siHot_400_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[2],color=color_blind_safe[2])
plt.errorbar(av_boundlow_Ge600, av_Int_Ge600/1e8, xerr=SE_siHot_600_boundary_low, yerr= SE_siHot_600_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[3],color=color_blind_safe[3])
plt.errorbar(av_boundlow_Ge800, av_Int_Ge800/1e8, xerr=SE_siHot_800_boundary_low, yerr= SE_siHot_800_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[4],color=color_blind_safe[4])
plt.errorbar(av_boundlow_Ge1000, av_Int_Ge1000/1e8, xerr=SE_siHot_1000_boundary_low, yerr= SE_siHot_1000_rint, markersize=12, capsize=7, mfc='none',mec=color_blind_safe[8],color=color_blind_safe[8])


#Linear fit
from scipy import stats

mnG = min(all_boxes_ge_lowerBoundary)
mxG = max(all_boxes_ge_lowerBoundary)

mnS = min(all_boxes_si_lowerBoundary)
mxS = max(all_boxes_si_lowerBoundary)

#SiGeSi

polyfit_fit_Si = np.polyfit(all_boxes_si_lowerBoundary, all_boxes_si_IntR, 1) 
polyfit_fit_Si = np.poly1d(polyfit_fit_Si)
xp_Si = np.linspace(mnS-0.001, mxS+0.001, 20)  
slope_intercept_Si = np.polyfit(all_boxes_si_lowerBoundary, all_boxes_si_IntR, 1)
#print('Slope, Intercept of Si polyfit numpy:',slope_intercept_Si)


#GeSiGe

polyfit_fit_Ge = np.polyfit(all_boxes_ge_lowerBoundary, all_boxes_ge_IntR, 1) 
polyfit_fit_Ge = np.poly1d(polyfit_fit_Ge)
#print('Polyfit Ge:',polyfit_fit_Ge)
xp_Ge = np.linspace(mnG-0.001, mxG+0.001,20)  
slope_intercept_Ge = np.polyfit(all_boxes_ge_lowerBoundary, all_boxes_ge_IntR, 1)
#print('Slope, Intercept of Ge polyfit numpy:',slope_intercept_Ge)


bbox = dict(boxstyle ="round", fc ="0.9") 


plt.plot(xp_Si, polyfit_fit_Si(xp_Si), '--', color ='blue', label = 'Ge(hot)')
plt.plot(xp_Ge, polyfit_fit_Ge(xp_Ge), '--', color ='black', label = 'Si(hot)')
plt.text(335, 2.7,"y=%.6fx+%.6f"%(polyfit_fit_Si[1],polyfit_fit_Si[0]))
plt.text(335, 3.7,"y=%.6fx+%.6f"%(polyfit_fit_Ge[1],polyfit_fit_Ge[0]))


#using scipy
res_Si = stats.linregress(all_boxes_si_lowerBoundary, all_boxes_si_IntR)
print(f"R-squared Si scipy: {res_Si.rvalue**2:.6f}")
print(f"Intercept Si scipy: {res_Si.intercept:.6f}")
print(f"Slope Si scipy: {res_Si.slope:.6f}")

res_Ge = stats.linregress(all_boxes_ge_lowerBoundary, all_boxes_ge_IntR)
print(f"R-squared Ge scipy: {res_Ge.rvalue**2:.6f}")
print(f"Intercept Ge scipy: {res_Ge.intercept:.6f}")
print(f"Slope Ge scipy: {res_Ge.slope:.6f}")

ax.annotate('$R^2$ Ge(hot): = %.6f' % (res_Si.rvalue**2),
            xy=(335, 3.0), xytext=(335, 3.0), bbox = bbox)

ax.annotate('$R^2$ Si(hot): = %.6f' % (res_Ge.rvalue**2),
            xy=(335, 4.0), xytext=(335, 4.0), bbox = bbox)

#Linear fit

plt.legend(bbox_to_anchor=(1.1, 1.0), loc='upper left',labelspacing=0.6, fontsize=20) 
plt.ylabel('\n\n $R_{int}$ ($10^8$ K/W)',fontsize=27)
plt.xlabel('\n\n Bottom Boundary Temperature (K)',fontsize=27)


plt.show()  

In [ ]:
#Size vs TempD

In [ ]:
##### from mpl_toolkits.mplot3d import axes3d, Axes3D
#from scipy.interpolate import griddata


fig, ax = plt.subplots(figsize=(12,8))


xline_si = rad_size
yline_si = TEMP_SI_ALL
zline_si = INT_R_SI_ALL
xline_si = np.array(xline_si); yline_si = np.array(yline_si); zline_si = np.array(zline_si);  

xline = rad_sizeG
yline = TEMP_GE_ALL
zline = INT_R_GE_ALL
xline = np.array(xline); yline = np.array(yline); zline = np.array(zline);  

av_size_Si200 = np.mean(xline_si[:][0])
av_Int_Si200 = np.mean(zline_si[:][0])
av_TempD_Si200 = np.mean(yline_si[:][0])
av_size_Si400 = np.mean(xline_si[:][1])
av_Int_Si400 = np.mean(zline_si[:][1])
av_TempD_Si400 = np.mean(yline_si[:][1])
av_size_Si600 = np.mean(xline_si[:][2])
av_Int_Si600 = np.mean(zline_si[:][2])
av_TempD_Si600 = np.mean(yline_si[:][2])
av_size_Si800 = np.mean(xline_si[:][3])
av_Int_Si800 = np.mean(zline_si[:][3])
av_TempD_Si800 = np.mean(yline_si[:][3])
av_size_Si1000 = np.mean(xline_si[:][4])
av_Int_Si1000 = np.mean(zline_si[:][4])
av_TempD_Si1000 = np.mean(yline_si[:][4])

av_size_Ge200 = np.mean(xline[:][0])
av_Int_Ge200 = np.mean(zline[:][0])
av_TempD_Ge200 = np.mean(yline[:][0])
av_size_Ge400 = np.mean(xline[:][1])
av_Int_Ge400 = np.mean(zline[:][1])
av_TempD_Ge400 = np.mean(yline[:][1])
av_size_Ge600 = np.mean(xline[:][2])
av_Int_Ge600 = np.mean(zline[:][2])
av_TempD_Ge600 = np.mean(yline[:][2])
av_size_Ge800 = np.mean(xline[:][3])
av_Int_Ge800 = np.mean(zline[:][3])
av_TempD_Ge800 = np.mean(yline[:][3])
av_size_Ge1000 = np.mean(xline[:][4])
av_Int_Ge1000 = np.mean(zline[:][4])
av_TempD_Ge1000 = np.mean(yline[:][4])


#Ge(hot)
plt.scatter(av_size_Si200, av_TempD_Si200,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="o",s=100,label='Ge(hot)-200')
plt.scatter(av_size_Si400, av_TempD_Si400,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="o",s=100,label='Ge(hot)-400')
plt.scatter(av_size_Si600, av_TempD_Si600,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="o",s=100,label='Ge(hot)-600')
plt.scatter(av_size_Si800, av_TempD_Si800,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="o",s=100,label='Ge(hot)-800')
plt.scatter(av_size_Si1000, av_TempD_Si1000,facecolors=color_blind_safe[8], edgecolors=color_blind_safe[8],marker="o",s=100,label='Ge(hot)-1000')


#Si(hot)
plt.scatter(av_size_Ge200, av_TempD_Ge200,facecolors=color_blind_safe[1], edgecolors=color_blind_safe[1],marker="s",s=100,label='Si(hot)-200')
plt.scatter(av_size_Ge400, av_TempD_Ge400,facecolors=color_blind_safe[2], edgecolors=color_blind_safe[2],marker="s",s=100,label='Si(hot)-400')
plt.scatter(av_size_Ge600, av_TempD_Ge600,facecolors=color_blind_safe[3], edgecolors=color_blind_safe[3],marker="s",s=100,label='Si(hot)-600')
plt.scatter(av_size_Ge800, av_TempD_Ge800,facecolors=color_blind_safe[4], edgecolors=color_blind_safe[4],marker="s",s=100,label='Si(hot)-800')
plt.scatter(av_size_Ge1000, av_TempD_Ge1000,facecolors=color_blind_safe[8], edgecolors=color_blind_safe[8],marker="s",s=100,label='Si(hot)-1000')

#Linear fit
from scipy import stats

mnG = min(all_boxes_ge_size)
mxG = max(all_boxes_ge_size)

mnS = min(all_boxes_si_size)
mxS = max(all_boxes_si_size)

#SiGeSi
#using numpy
p30_Si = np.poly1d(np.polyfit(all_boxes_si_size, all_boxes_si_TempD, 3))
xp_p30_Si = np.linspace(-0.001, mxS+0.001,5)
slope_intercept_p30_Si = np.polyfit(all_boxes_si_size, all_boxes_si_TempD, 1)
#print('Slope, Intercept of p30_Si numpy:',slope_intercept)


polyfit_fit_Si = np.polyfit(all_boxes_si_size, all_boxes_si_TempD, 1) 
polyfit_fit_Si = np.poly1d(polyfit_fit_Si)
xp_Si = np.linspace(mnS-0.001, mxS+0.001, 20)  
slope_intercept_Si = np.polyfit(all_boxes_si_size, all_boxes_si_TempD, 1)
#print('Slope, Intercept of Si polyfit numpy:',slope_intercept_Si)


#GeSiGe
#using numpy
p30_Ge = np.poly1d(np.polyfit(all_boxes_ge_size, all_boxes_ge_TempD, 3))
xp_p30_Ge = np.linspace(-0.001, mxG+0.001,5)
slope_intercept_p30_Ge = np.polyfit(all_boxes_ge_size, all_boxes_ge_TempD, 1)
#print('Slope, Intercept of p30_Ge numpy:',slope_intercept)


polyfit_fit_Ge = np.polyfit(all_boxes_ge_size, all_boxes_ge_TempD, 1) 
polyfit_fit_Ge = np.poly1d(polyfit_fit_Ge)
#print('Polyfit Ge:',polyfit_fit_Ge)
xp_Ge = np.linspace(mnG-0.001, mxG+0.001,20)  #xp = np.linspace(-0.001, mx+0.001, 5)
slope_intercept_Ge = np.polyfit(all_boxes_ge_size, all_boxes_ge_TempD, 1)
#print('Slope, Intercept of Ge polyfit numpy:',slope_intercept_Ge)


bbox = dict(boxstyle ="round", fc ="0.9") 


plt.plot(xp_Si, polyfit_fit_Si(xp_Si), '--', color ='blue', label = 'Ge(hot)')
plt.plot(xp_Ge, polyfit_fit_Ge(xp_Ge), '--', color ='black', label = 'Si(hot)')
plt.text(400, 68,"y=%.6fx+%.6f"%(polyfit_fit_Si[1],polyfit_fit_Si[0]))
plt.text(110, 168,"y=%.6fx+%.6f"%(polyfit_fit_Ge[1],polyfit_fit_Ge[0]))


#using scipy
res_Si = stats.linregress(all_boxes_si_size, all_boxes_si_TempD)
print(f"R-squared Si scipy: {res_Si.rvalue**2:.6f}")
print(f"Intercept Si scipy: {res_Si.intercept:.6f}")
print(f"Slope Si scipy: {res_Si.slope:.6f}")

res_Ge = stats.linregress(all_boxes_ge_size, all_boxes_ge_TempD)
print(f"R-squared Ge scipy: {res_Ge.rvalue**2:.6f}")
print(f"Intercept Ge scipy: {res_Ge.intercept:.6f}")
print(f"Slope Ge scipy: {res_Ge.slope:.6f}")

ax.annotate('$R^2$ Ge(hot): = %.6f' % (res_Si.rvalue**2),
            xy=(400, 60), xytext=(400, 60), bbox = bbox)

ax.annotate('$R^2$ Si(hot): = %.6f' % (res_Ge.rvalue**2),
            xy=(110, 160), xytext=(110, 160), bbox = bbox)

#Linear fit

plt.legend(bbox_to_anchor=(1.1, 1.0), loc='upper left',labelspacing=0.6, fontsize=20) 
plt.xlabel('\n\n Length (nm)',fontsize=27)
plt.ylabel('\n\n Temperature difference (K)',fontsize=27)

plt.xticks([110.0, 219.9, 329.9, 439.9, 550.0],
           ['110.0','219.9','329.9','439.9','550.0'],rotation=0) 


plt.show()  